## 144_5 dashboard_Maturity
* [144_5 dashboard](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/144_5%20dashboard.ipynb)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2026-01-04 14:42:56


In [2]:
from pathlib import Path
import shutil
from datetime import datetime

def save_dashboard(html_out, OUTPUT_DIR, PROJECT_NAME):
    out_dir = Path(OUTPUT_DIR)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Create timestamped filename
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    timestamped_path = out_dir / f"{PROJECT_NAME}_dashboard_{stamp}.html"

    # Create "latest" filename (always the same)
    latest_path = out_dir / "dashboard.html"

    # Write HTML to timestamped file
    with open(timestamped_path, "w", encoding="utf-8") as f:
        f.write(html_out)

    # Copy to dashboard.html
    shutil.copyfile(timestamped_path, latest_path)

    print(f"Saved (timestamped): {timestamped_path}")
    print(f"Updated (latest): {latest_path}")
    print("Current path:", Path.cwd())

    return str(timestamped_path)

In [3]:
# ---------------------- Build HTML dashboard --------------------------------

import html
import pandas as pd
from datetime import datetime
import time,os,sys,re
from pathlib import Path 
import shutil



def make_sat_dashboard(
    OUTPUT_DIR: str,
    PROJECT_NAME: str,
    stamp: str,
    *,
    map_gallery: list[dict] | None = None,
    intro_html: str = "",                 
    charts_blocks: list[dict] | None = None,
    summary_html: str = "",
    user_cases: list[dict] | None = None,
    current_level: int = 1  # <— 🔹 Ny parameter för roadmap-status
) -> str:
    gallery_html = _render_map_gallery_html(map_gallery or [])
    intro_block  = _render_intro_html(intro_html)              
    charts_html  = _render_charts_html(charts_blocks or [])    
    user_cases_html = _render_user_cases_html(user_cases or []) 

    # ---------------- Helper: create roadmap section ----------------
    def _render_roadmap_html(level: int) -> str:
        progress_percent = int((level / 5) * 100)
        if level <= 1:
            status_color = "#ef4444"  # 🔴 röd
        elif level == 2:
            status_color = "#f59e0b"  # 🟠 orange
        elif level == 3:
            status_color = "#facc15"  # 🟡 gul
        elif level == 4:
            status_color = "#22c55e"  # 🟢 grön
        else:
            status_color = "#2563eb"  # 🔵 blå (klar)

        return f"""
        <!-- Collapsible Digital Twin Roadmap -->
        <section style="
          font-family:system-ui,Segoe UI,Roboto,sans-serif;
          margin:16px 0;
          border:1px solid #ccc;
          border-radius:8px;
          overflow:hidden;
          box-shadow:0 2px 6px rgba(0,0,0,0.15);
          background:white;">

          <!-- Header -->
          <div onclick="toggleRoadmap()" 
               style="background:linear-gradient(135deg,#1e3a8a 0%,#2563eb 100%);
                      color:white; padding:16px 20px;
                      cursor:pointer; font-weight:700; font-size:1.1rem;
                      display:flex; align-items:center; justify-content:space-between;">
              
              <div>🧭 Digital Twin Roadmap</div>

              <div style="display:flex; align-items:center; gap:12px;">
                <div style="background:{status_color}; color:black; padding:4px 10px; 
                            border-radius:20px; font-size:0.85rem; font-weight:600;">
                  Nivå {level} / 5
                </div>
                <span id='roadmap_arrow' style="font-size:1.3rem; transition:transform 0.3s;">▼</span>
              </div>
          </div>

          <!-- Content -->
          <div id="roadmap_body" 
               style="display:none; opacity:0; transform:translateY(-10px); 
                      transition:opacity 0.5s ease, transform 0.5s ease;
                      padding:20px; color:#111;">

            <p style="font-size:1rem; line-height:1.6;">
              En visuell resa genom 
              <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/241'
                 target='_blank' style='color:#2563eb; text-decoration:none; font-weight:600;'>
                 Hiking Maturity Model</a> – 
              från kaos till smart data och levande digitala tvillingar - <a href="https://en.wikipedia.org/wiki/Single_source_of_truth">SSOT</a> - <a href=https://www.youtube.com/watch?v=NKDyRAtjvDI">video "Från kaos till smart data" </a>.
            </p>

            <!-- Timeline -->
            <div style="display:flex; justify-content:space-between; flex-wrap:wrap; gap:16px; text-align:center;">
              <div style="flex:1; min-width:150px;"><div style="font-size:2rem;">🌀</div><h3>Nivå 0 → 1</h3><p>Från kaos till webb.<br>📍 Leden synlig på OSM & Wikidata.</p></div>
              <div style="flex:1; min-width:150px;"><div style="font-size:2rem;">🌍</div><h3>Nivå 2</h3><p>Öppna data publiceras.<br>💾 GeoJSON & CSV på GitHub.</p></div>
              <div style="flex:1; min-width:150px;"><div style="font-size:2rem;">🧩</div><h3>Nivå 3</h3><p>Standardiserad data.<br>📐 INSPIRE, Schema.org, Wikidata.</p></div>
              <div style="flex:1; min-width:150px;"><div style="font-size:2rem;">🔗</div><h3>Nivå 4</h3><p>Länkad data & flerspråkighet.<br>🌐 <a href=https://vimeo.com/36752317>SameAs-länkar</a>.</p></div>
              <div style="flex:1; min-width:150px;"><div style="font-size:2rem;">🤖</div><h3>Nivå 5</h3><p>Roundtripping, <a href="https://en.wikipedia.org/wiki/Single_source_of_truth">SSOT</a> & smart data.<br>🏁 SAT Digital Twin v1.</p></div>
            </div>

            <!-- Progressbar -->
            <div style="
              position:relative;
              height:6px;
              background:#ddd;
              margin:32px 10%;
              border-radius:3px;">
              <div style="
                position:absolute;
                left:0;
                top:0;
                height:100%;
                width:{progress_percent}%;
                background:{status_color};
                border-radius:3px;
                transition:width 0.6s;">
              </div>
            </div>

            <p style="font-size:0.9rem; margin-top:-8px; text-align:center;">
              🔸 Aktuell mognadsnivå: <b>{level}/5</b>  
              – varje steg tar oss närmare en komplett digital tvilling av leden.
            </p>
          </div>

          <!-- Script -->
          <script>
            function toggleRoadmap() {{
              var body = document.getElementById('roadmap_body');
              var arrow = document.getElementById('roadmap_arrow');
              if (body.style.display === 'none' || body.style.display === '') {{
                body.style.display = 'block';
                setTimeout(() => {{
                  body.style.opacity = 1;
                  body.style.transform = 'translateY(0)';
                  arrow.style.transform = 'rotate(180deg)';
                }}, 50);
              }} else {{
                body.style.opacity = 0;
                body.style.transform = 'translateY(-10px)';
                arrow.style.transform = 'rotate(0deg)';
                setTimeout(() => {{
                  body.style.display = 'none';
                }}, 500);
              }}
            }}
          </script>
        </section>
        """
    # -----------------------------------------------------------------
    gallery_html = _render_map_gallery_html(map_gallery or [])
    intro_block  = _render_intro_html(intro_html)
    charts_html  = _render_charts_html(charts_blocks or [])
    user_cases_html = _render_user_cases_html(user_cases or [])
    roadmap_html = _render_roadmap_html(current_level)  # <— Lägg till roadmap här

    # -----------------------------------------------------------------
        
    html_out = f"""
<!doctype html>
<html lang="en">
<head>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
  <meta charset="utf-8">
  <title>Stockholm Archipelago Trail — An Open Data Journey</title>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
  <style>
    body{{margin:0;padding:20px;background:#f8fafc;font:14px/1.4 system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;color:#111827}}
    .sat-wrap{{max-width:1200px;margin:0 auto}} 
    .sat-hero {{
      background: linear-gradient(135deg, #1e3a8a 0%, #2563eb 100%);
      color: white;
      border-radius: 14px;
      padding: 28px 26px 22px 26px;
      margin-bottom: 18px;
      display: flex;
      flex-wrap: wrap;
      align-items: flex-start;
      justify-content: space-between;
      box-shadow: 0 3px 10px rgba(0,0,0,0.2);
      position: relative;
    }}
    
    .sat-hero h1 {{
      margin: 0;
      font-size: 1.8rem;
      font-weight: 700;
      letter-spacing: 0.3px;
      line-height: 1.3;
    }}
    
    .sat-hero h1 .sub {{
      font-weight: 400;
      opacity: 0.9;
    }}
    
    .sat-hero .tagline {{
      margin: 6px 0 0 0;
      font-size: 1rem;
      opacity: 0.85;
      line-height: 1.4;
      max-width: 720px;
    }}

.hero-updated {{
  background: rgba(255,255,255,0.15);
  padding: 4px 10px;
  border-radius: 6px;
  font-size: 0.85rem;
  letter-spacing: 0.3px;
  font-weight: 500;
  align-self: flex-start;
}}

.hero-updated:hover {{
  background: rgba(255,255,255,0.25);
}}
    .sat-hero{{background:#111827;border-radius:14px;color:white;padding:16px 18px;margin-bottom:16px;display:flex;flex-wrap:wrap;align-items:center;gap:10px}}
    .sat-hero h1{{margin:0;font-size:20px}}
    .sat-hero a{{color:#a5b4fc;text-decoration:none}}
    .sat-row{{display:grid;gap:14px;grid-template-columns:1fr}}
    @media(min-width:960px){{ .sat-row{{grid-template-columns:2fr 1fr}} }}
    .sat-panel{{background:#fff;border-radius:12px;box-shadow:0 2px 10px rgba(0,0,0,.06);padding:12px}}
    .muted{{color:#6b7280}}
    .sat-card{{ position:relative; }}
    .issues{{ position:absolute; top:10px; right:10px; display:flex; gap:6px; align-items:center; z-index:2; }}
    .issue-icon{{ font-size:18px; line-height:1; color:#374151; text-decoration:none; }}
    .issue-icon:hover{{ color:#111827; }}
    .issue-badge{{
      font:600 12px/1 system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;
      padding:4px 6px; border-radius:999px; background:#f3f4f6; color:#111827; text-decoration:none; border:1px solid #e5e7eb;
    }}
    .issue-badge:hover{{ background:#e5e7eb; }}
  </style>
</head>
<body>

    {intro_block}     
    {roadmap_html}
    {gallery_html}
    {user_cases_html} 
    <div class="sat-row">
      <div class="sat-panel">
        {charts_html}   
      </div>
      <div class="sat-panel">
        {summary_html}
      </div>
    </div>
  </div>
  </div>
  </div>
<script>
  // === FILTER TAGS SCRIPT START ===
  function updateFilters() {{
    const active = Array.from(document.querySelectorAll('.uc-tag.active'))
                        .map(t => t.dataset.tag.toLowerCase());

    document.querySelectorAll('.uc-card').forEach(card => {{
      const tags = card.dataset.tags.toLowerCase();
      if (active.length === 0 || active.some(a => tags.includes(a))) {{
        card.style.display = 'block';
      }} else {{
        card.style.display = 'none';
      }}
    }});
  }}

  function toggleTag(tagText) {{
    const selector = `.uc-tag[data-tag="${{tagText}}"]`;
    document.querySelectorAll(selector).forEach(tagEl => {{
      tagEl.classList.toggle('active');
    }});
    updateFilters();
  }}

  document.addEventListener('click', e => {{
    if (e.target.classList.contains('uc-tag')) {{
      const tagText = e.target.dataset.tag;
      toggleTag(tagText);
    }}
    if (e.target.id === 'reset-filters') {{
      document.querySelectorAll('.uc-tag.active').forEach(t => t.classList.remove('active'));
      updateFilters();
    }}
  }});
  // === FILTER TAGS SCRIPT END ===
</script>
</body>
</html>
"""
    return save_dashboard(html_out, OUTPUT_DIR, PROJECT_NAME)

def _render_intro_html(intro_html: str, mode: str = "on") -> str:
    """
    mode:
      - "off"         -> render nothing
      - "inline"      -> tiny inline link bar (no big panel, minimal spacing)
      - "collapsible" -> <details> with a 'Links' summary (closed by default)
      - "panel"       -> compact panel (if you still want a box)
    """
    if not intro_html or mode == "off":
        return ""

    if mode == "inline":
        return f'''
        <div class="sat-links" style="margin:4px 0 6px; font-size:.92rem;
             display:flex; flex-wrap:wrap; gap:.5rem; align-items:center;">
          {intro_html}
        </div>
        <style>
          .sat-links a {{ color:#2563eb; text-decoration:none }}
          .sat-links a:hover {{ text-decoration:underline }}
          .sat-links p {{ margin:0 }}
        </style>
        '''

    if mode == "collapsible":
        return f'''
        <details class="sat-links" style="margin:2px 0">
          <summary style="cursor:pointer; font-weight:600">Links</summary>
          <div style="margin-top:6px; display:flex; flex-wrap:wrap; gap:.5rem;">
            {intro_html}
          </div>
        </details>
        <style>
          .sat-links a {{ color:#2563eb; text-decoration:none }}
          .sat-links a:hover {{ text-decoration:underline }}
          .sat-links p {{ margin:.2rem 0 }}
        </style>
        '''

    # fallback: compact panel (kept for completeness)
    return f'''
    <section class="sat-panel" style="padding:6px 0">
      <div class="sat-intro">{intro_html}</div>
    </section>
    <style>
      .sat-intro p{{margin:.2rem 0}}
      .sat-intro a{{color:#2563eb;text-decoration:none}}
      .sat-intro a:hover{{text-decoration:underline}}
    </style>
    '''

def _render_charts_html(charts: list[dict]) -> str:
    if not charts:
        return ""
    import html as _h, re

    def _badge(url: str) -> str:
        # Try to extract issue number to show like #142
        m = re.search(r"/issues/(\d+)", url)
        label = f"#{m.group(1)}" if m else "Issue"
        u = _h.escape(url)
        return f'<a class="issue-badge" href="{u}" target="_blank" rel="noopener">{label}</a>'

    out = []
    for ch in charts:
        title  = _h.escape(ch.get("title", ""))
        body   = ch.get("body_html", "")
        issues = ch.get("issues") or []  # list of URLs (strings)
        badges = " ".join(_badge(u) for u in issues)

        out.append(f"""
        <section class="sat-panel">
          <div class="chart-head">
            <h3 class="chart-title">{title}</h3>
            {'<div class="issue-badges">'+badges+'</div>' if badges else ''}
          </div>
          <div class="chart-desc">{body}</div>
        </section>
        """)

    return """
    <div class="charts-col">
      {}
    </div>
    <style>
      .charts-col > .sat-panel + .sat-panel{{margin-top:12px}}

      .chart-head{{display:flex;align-items:center;gap:8px;flex-wrap:wrap}}
      .chart-title{{margin:0;font-size:16px;font-weight:700}}

      .issue-badge{{
        display:inline-block; padding:2px 8px; border-radius:999px;
        background:#eef2ff; color:#3730a3; font-size:12px; font-weight:600;
        text-decoration:none; border:1px solid #c7d2fe;
      }}
      .issue-badge:hover{{background:#e0e7ff}}
      .chart-desc{{color:#374151}}
      .chart-desc p{{margin:.4rem 0}}
      .chart-desc a{{color:#2563eb;text-decoration:none}}
      .chart-desc a:hover{{text-decoration:underline}}
      .related-videos {{
          border-top: 1px solid #e5e7eb;
          margin-top: 10px;
          padding-top: 6px;
        }}
        
        .related-videos summary {{
          cursor: pointer;
          font-weight: 600;
          color: #1e3a8a;
          margin-top: 4px;
          display: flex;
          align-items: center;
          gap: 6px;
        }}
        
        .related-videos summary::after {{
          content: "▼";
          font-size: 11px;
          color: #6b7280;
          transition: transform 0.2s ease;
        }}
        
        .related-videos[open] summary::after {{
          transform: rotate(-180deg);
        }}
        
        .related-videos details {{
          margin-top: 4px;
        }}
        
        .related-videos ul {{
          list-style: none;
          margin: 6px 0 0 0;
          padding: 0;
        }}
        
        .related-videos .video-item {{
          margin-bottom: 6px;
          font-size: 13px;
          line-height: 1.4;
        }}
        
        .related-videos .video-item a {{
          color: #2563eb;
          text-decoration: none;
        }}
        
        .related-videos .video-item a:hover {{
          text-decoration: underline;
        }}
        
        .related-videos .muted {{
          color: #6b7280;
          font-size: 12px;
        }}


    </style>
    """.format("".join(out))
    

In [4]:
def _render_map_gallery_html(items: list[dict]) -> str:
    """
    Render a responsive gallery of map links with clickable filters and synced tag highlighting.
    """
    if not items:
        return ""

    import html as _h, re
    def _esc(v): return _h.escape("" if v is None else str(v))

    # --- Issue badges helper ---
    def _issue_badges(issues: list[str] | None) -> str:
        if not issues:
            return ""
        badges = []
        for u in issues:
            m = re.search(r"/issues/(\d+)", u or "")
            num = m.group(1) if m else None
            label = f"#{num}" if num else "Issue"
            title = f"Issue #{num}" if num else "GitHub Issue"
            badges.append(
                f'<a class="issue-badge" href="{_esc(u)}" target="_blank" rel="noopener" '
                f'title="{_esc(title)}">{_esc(label)}</a>'
            )
        badges.insert(0, f'<a class="issue-icon" href="{_esc(issues[0])}" target="_blank" '
                        f'rel="noopener" title="GitHub Issues"><i class="fab fa-github"></i></a>')
        return f'<div class="issues">{"".join(badges)}</div>'

    # --- Collect unique buzzwords + stakeholders ---
    buzzwords = sorted({bw for it in items for bw in it.get("buzzwords", [])})
    stakeholders = sorted({st for it in items for st in it.get("stakeholders", [])})

    def chip(label, tag, kind):
        return f'<button class="filter-chip {kind}" data-tag="{_esc(tag)}">{_esc(label)}</button>'

    # --- Filter bar ---
    buzz_html = "".join(chip(b, b, "buzz") for b in buzzwords)
    stake_html = "".join(chip(s, s, "stakeholder") for s in stakeholders)
    filter_html = f"""
    <div class="filter-bar">
      <div class="filter-section"><strong>🏷️ Buzzwords:</strong> {buzz_html}</div>
      <div class="filter-section"><strong>👥 Stakeholders:</strong> {stake_html}</div>
      <button class="filter-clear" onclick="clearFilters()">Rensa filter</button>
    </div>
    """

    cards = []
    for it in items:
        title = _esc(it.get("title", ""))
        url = _esc(it.get("url", ""))
        img1x = _esc(it.get("img", ""))
        img2x = _esc(it.get("img2x", "")) or img1x.replace("400x225", "800x450")
        desc_raw = it.get("desc", "")
        desc_html = desc_raw if it.get("desc_is_html", False) else _esc(desc_raw)
        buzz = it.get("buzzwords", [])
        stake = it.get("stakeholders", [])
        tags_for_data = ",".join(buzz + stake)

        buzz_tags_html = "".join(
            f'<span class="tag buzz" data-tag="{_esc(t)}" onclick="toggleFilterTag(\'{_esc(t)}\')">{_esc(t)}</span>'
            for t in buzz
        )
        stake_tags_html = "".join(
            f'<span class="tag stakeholder" data-tag="{_esc(s)}" onclick="toggleFilterTag(\'{_esc(s)}\')">{_esc(s)}</span>'
            for s in stake
        )

        # --- ID badge ---
        id_html = ""
        id_value = it.get("id")
        if id_value:
            prefix = id_value.split("-")[0].lower()
            id_anchor = f"SAT_ALL_IN_ONE_142_3_dashboard_latest.html#{_esc(id_value)}"
            id_html = f"""
              <a class="id-badge {prefix}" href="{id_anchor}" title="Anchor to {id_value}">
                📍 {id_value}
              </a>
            """

        issues_html = _issue_badges(it.get("issues"))

        # --- Related videos (if any) ---
        videos_html = ""
        if it.get("related_videos"):
            video_cards = []
            for v in it["related_videos"]:
                title_v = _esc(v.get("title", ""))
                url_v = _esc(v.get("url", ""))
                lang_v = _esc(v.get("lang", ""))
                comment_v = _esc(v.get("comment", ""))
                video_cards.append(f"""
                  <li class="video-item">
                    🎬 <a href="{url_v}" target="_blank" rel="noopener">{title_v}</a>
                    <span class="muted">[{lang_v}]</span><br/>
                    <small>{comment_v}</small>
                  </li>
                """)
            videos_html = f"""
              <div class="related-videos">
                <details class="related-videos">
                  <summary>🎥 Relaterade filmer ({len(video_cards)}) — klicka för att visa</summary>
                  <ul>{"".join(video_cards)}</ul>
                </details>
              </div>
            """

        # --- Image block ---
        media_html = (
            f"""
              <a class="gallery-card-link" href="{url}" target="_blank" rel="noopener">
                <picture>
                  <source srcset="{img1x} 1x, {img2x} 2x" type="image/jpeg">
                  <img class="gallery-media" src="{img1x}"
                       srcset="{img1x} 1x, {img2x} 2x"
                       width="400" height="225" loading="lazy" decoding="async"
                       alt="{title}">
                </picture>
              </a>
            """ if img1x else f"""
              <a class="gallery-fallback" href="{url}" target="_blank" rel="noopener">
                <div class="gallery-fallback-title">{title}</div>
              </a>
            """
        )

        # --- Full card (now includes videos_html correctly) ---
        card = f"""
        <article class="sat-card" data-tags="{_esc(tags_for_data)}">
          {id_html}
          {issues_html}
          {media_html}
          <div class="gallery-meta">
            <a class="gallery-title" href="{url}" target="_blank">{title}</a>
            {f'<div class="tag-row buzzwords">{buzz_tags_html}</div>' if buzz_tags_html else ''}
            {f'<div class="tag-row stakeholders">{stake_tags_html}</div>' if stake_tags_html else ''}
            <div class="gallery-desc">{desc_html}</div>
            {videos_html}
          </div>
        </article>
        """
        cards.append(card)


    return f"""
    <section class="sat-panel">
      <div class="gallery-header">
        <h2>Kartgalleri</h2>
        <div class="muted">Länkar till senaste skapade kartorna</div>
      </div>
      {filter_html}
      <div class="sat-gallery">{''.join(cards)}</div>
    </section>

    <style>
      .sat-gallery {{display:grid;gap:12px;grid-template-columns:1fr}}
      @media(min-width:680px){{.sat-gallery{{grid-template-columns:repeat(2,1fr)}}}}
      @media(min-width:1024px){{.sat-gallery{{grid-template-columns:repeat(3,1fr)}}}}

      .sat-card{{position:relative;background:#fff;border-radius:12px;box-shadow:0 2px 10px rgba(0,0,0,.06);
                 padding:10px;display:flex;flex-direction:column;gap:8px}}
      .gallery-media{{display:block;width:100%;border-radius:10px;object-fit:cover}}

      /* --- Titel med blå gradientband --- */
      .gallery-title {{
        font-weight:700;
        font-size:16px;
        color:#1e3a8a;
        text-decoration:none;
        display:inline-block;
        margin-bottom:4px;
        line-height:1.3;
        background:linear-gradient(90deg,#e0f2fe,#bfdbfe);
        padding:4px 8px;
        border-radius:8px;
        box-shadow:0 1px 3px rgba(0,0,0,0.08);
      }}
      .gallery-title:hover {{
        background:linear-gradient(90deg,#bfdbfe,#93c5fd);
        text-decoration:underline;
      }}

.id-badge {{
  position: absolute;
  top: 10px;
  left: 10px;
  background: linear-gradient(135deg, #e0f2fe, #bfdbfe);
  color: #1e3a8a;
  font-weight: 700;
  font-size: 12px;
  padding: 4px 8px;
  border-radius: 999px;
  border: 1px solid #93c5fd;
  box-shadow: 0 2px 6px rgba(0,0,0,0.1);
  text-decoration: none;
  display: flex;
  align-items: center;
  gap: 4px;
}}
.id-badge::before {{
  content: "📎";
  font-size: 13px;
}}
.id-badge:hover {{
  background: linear-gradient(135deg, #bfdbfe, #93c5fd);
  color: #1e3a8a;
}}

      .tag {{font-size:12px;padding:2px 8px;border-radius:999px;border:1px solid #e5e7eb;
             cursor:pointer;transition:.15s}}
      .tag:hover {{opacity:0.8;transform:scale(1.05)}}
      .tag.active {{filter:brightness(0.9);transform:scale(1.05)}}
      .tag.buzz {{background:#eef2ff;color:#3730a3;border-color:#c7d2fe;}}
      .tag.stakeholder {{background:#d1fae5;color:#065f46;border-color:#a7f3d0;}}
      .tag.buzz.active{{background:#2563eb;color:white}}
      .tag.stakeholder.active{{background:#059669;color:white}}

      /* --- Filter chip active styles --- */
      .filter-chip.buzz {{background:#eef2ff;color:#3730a3;border-color:#c7d2fe;}}
      .filter-chip.buzz.active {{background:#2563eb;color:#fff;border-color:#2563eb;}}
      .filter-chip.stakeholder {{background:#d1fae5;color:#065f46;border-color:#a7f3d0;}}
      .filter-chip.stakeholder.active {{background:#059669;color:#fff;border-color:#059669;}}
    </style>

    <script>
      function clearFilters() {{
        document.querySelectorAll('.filter-chip').forEach(c => c.classList.remove('active'));
        document.querySelectorAll('.tag').forEach(t => t.classList.remove('active'));
        document.querySelectorAll('.sat-card').forEach(c => c.style.display = '');
      }}
      document.querySelectorAll('.filter-chip').forEach(chip => {{
        chip.addEventListener('click', () => {{
          chip.classList.toggle('active');
          const tag = chip.getAttribute('data-tag');
          document.querySelectorAll(`.tag[data-tag="${{tag}}"]`).forEach(t => t.classList.toggle('active'));
          const active = Array.from(document.querySelectorAll('.filter-chip.active'))
                              .map(c => c.getAttribute('data-tag'));
          document.querySelectorAll('.sat-card').forEach(c => {{
            const tags = (c.getAttribute('data-tags') || '').split(',');
            const match = active.every(a => tags.includes(a));
            c.style.display = active.length ? (match ? '' : 'none') : '';
          }});
        }});
      }});

      // Klicka tag i kortet → aktivera chip + markera taggar
      function toggleFilterTag(tag) {{
        const chip = document.querySelector(`.filter-chip[data-tag="${{tag}}"]`);
        if (chip) chip.click();
        window.scrollTo({{ top: 0, behavior: "smooth" }});
      }}
    </script>
    """


In [5]:
def _render_user_cases_html(user_cases: list[dict]) -> str:
    if not user_cases:
        return ""

    def _tag(text):
        return f'<span class="uc-tag" data-tag="{text}" onclick="toggleFilterTag(\'{text}\')">{text}</span>'

    # Build index and cards
    index_links = []
    cards = []

    for i, case in enumerate(user_cases, start=1):
        uc_id = f"UC-{i}"
        title = case["title"]
        buzz = " ".join(_tag(b) for b in case.get("buzzwords", []))
        stakeholders = " ".join(_tag(s) for s in case.get("stakeholders", []))
        desc = case.get("desc", "")

        index_links.append(f'<a href="#{uc_id}" class="uc-index-link">{uc_id} {title}</a>')

        cards.append(f"""
        <div class="uc-card" id="{uc_id}">
          <div class="uc-header">
            <span class="uc-id">{uc_id}</span>
            <span class="uc-title">{title}</span>
          </div>
          <div class="uc-meta"><b>Keywords:</b> {buzz}</div>
          <div class="uc-meta"><b>Stakeholders:</b> {stakeholders}</div>
          <div class="uc-desc">{desc}</div>
          <div class="uc-back"><a href="#usercase-index">↑ Back to top</a></div>
        </div>
        """)

    return f"""
    <section class="sat-panel" id="usercase-index">
      <h2>User Cases</h2>
      <div class="uc-index">
        <b>Jump to:</b><br>
        {' | '.join(index_links)}
      </div>
      <div class="uc-grid">
        {''.join(cards)}
      </div>
    </section>

    <style>
      .uc-index {{
        margin-bottom:16px;
        font-size:14px;
        line-height:1.6;
      }}
      .uc-index-link {{
        color:#2563eb;
        text-decoration:none;
        font-weight:500;
      }}
      .uc-index-link:hover {{
        text-decoration:underline;
      }}
      .uc-grid {{
        display:grid;
        grid-template-columns:repeat(auto-fit, minmax(260px,1fr));
        gap:16px;
        margin-top:12px;
      }}
      .uc-card {{
        border:1px solid #e5e7eb;
        border-radius:12px;
        padding:14px;
        background:#fff;
        box-shadow:0 2px 4px rgba(0,0,0,0.05);
        scroll-margin-top:80px;
        display:flex;
        flex-direction:column;
        justify-content:space-between;
      }}

      /* Titel med gradientband (matchar kartgalleri) */
      .uc-header {{
        display:flex;
        align-items:center;
        gap:8px;
        background:linear-gradient(90deg,#e0f2fe,#bfdbfe);
        border-radius:8px;
        padding:6px 10px;
        margin-bottom:6px;
        box-shadow:0 1px 3px rgba(0,0,0,0.08);
      }}
      .uc-id {{
        font-weight:600;
        color:#1e3a8a;
        font-size:13px;
      }}
      .uc-title {{
        font-weight:700;
        font-size:15px;
        color:#1e3a8a;
      }}

      .uc-meta {{
        font-size:13px;
        color:#374151;
        margin-bottom:4px;
        display:flex;
        flex-wrap:wrap;
        gap:6px;
      }}
      .uc-tag {{
        cursor: pointer;
        transition: 0.15s;
        display:inline-block;
        padding:4px 10px;
        border-radius:999px;
        font-size:12px;
        font-weight:500;
        text-decoration:none;
      }}
      .uc-meta:nth-of-type(1) .uc-tag {{
        background:#eef2ff;
        color:#3730a3;
      }}
      .uc-meta:nth-of-type(2) .uc-tag {{
        background:#dcfce7;
        color:#166534;
      }}
      .uc-tag:hover {{
        transform: scale(1.05);
        opacity: 0.9;
      }}

      .uc-desc {{
        font-size:13px;
        line-height:1.5;
        margin-top:4px;
      }}

      .uc-back {{
        margin-top:8px;
        text-align:right;
      }}
      .uc-back a {{
        font-size:12px;
        color:#6b7280;
        text-decoration:none;
      }}
      .uc-back a:hover {{
        text-decoration:underline;
      }}

      .highlight {{
        outline:3px solid #2563eb;
        background:#eff6ff;
      }}
    </style>

    <script>
      const params = new URLSearchParams(window.location.search);
      const uc = params.get("uc");
      if (uc) {{
        const target = document.getElementById(uc);
        if (target) {{
          target.classList.add("highlight");
          target.scrollIntoView({{ behavior: "smooth", block: "center" }});
        }}
      }}

      // Synka med filter från kartgalleriet
      function toggleFilterTag(tag) {{
        const chip = document.querySelector(`.filter-chip[data-tag="${{tag}}"]`);
        if (chip) chip.click();
        window.scrollTo({{ top: document.querySelector('.sat-panel').offsetTop - 50, behavior: 'smooth' }});
      }}
    </script>
    """


In [6]:
# =================== Save helpers ===========================================
def _to_latest_name(filename: str | Path) -> Path:
    p = Path(filename)
    stem = p.stem
    new_stem = re.sub(r'_(?:20\d{6})(?:_\d{4})?$', '', stem)
    return p.with_name(new_stem + "_latest.html")

def save_with_latest(map_object, filename: str | Path):
    from pathlib import Path

    print("Current path:", Path.cwd())
    map_object.save(str(filename))
    latest_path = _to_latest_name(filename)
    print("Latest path ",latest_path)
    shutil.copyfile(filename, latest_path)
    print(f"Saved: {filename}\nUpdated: {latest_path}")


In [7]:
gallery = [   {
    "id": "VISITSWEDEN-2",
    "title": "🌍 SAT - Saltkråkan & den datadrivna skärgården",
    "url": "./SAT300_FilmingLocations.html",
    "img": "./thumbs/SAT300_FilmingLocations_400x225.jpg",
    "desc": "📽️ Kartan visar inspelningsplatser för filmen hämtat från <a href='https://w.wiki/FuJX' target=_blank>Wikidatas kunskapsgraf</a> <b>Vi på Saltkråkan</b> — en ikonisk del av svensk kultur som idag passeras av <b>Stockholm Archipelago Trail</b> (SAT).<br /><br />"
            "Genom att tänka <b>datadrivet</b> och bygga <b>digitala tvillingar</b> av skärgården kan vi skapa helt nya möjligheter: att se leder, naturreservat, servicepunkter och tillgänglighet i realtid. "
            "Filmen <a href='https://www.youtube.com/watch?v=pvGUv8Bf0Go&list=PLNWUKRLAYDeR22ntYxzR95rlW-0Ed4I7q&index=8' target='_blank'>“Datarevolutionen bakom kartor”</a> visar hur kartor blir "
            "en del av den digitala infrastrukturen — där varje plats, från Saltkråkan till minsta dass, kan beskrivas med öppna data.<br /><br />"
            "I <a href='https://www.facebook.com/groups/2875020699552247' target=_blank>FB-gruppen</a> ser vi tydligt behovet av ett digitalt ekosystem som gör det enkelt att veta: "
            "är dricksvattnet på, är toaletten tillgänglig, finns det varningar på leden, eller extra erbjudanden längs vägen? "
            "Idag saknas helheten — data finns utspridd och inte alltid öppen eller länkad.<br /><br />"
            "<b>För att bygga en digital tvilling av skärgården behövs:</b>"
            "<ul>"
            "<li>🧭 Öppna API:er med realtidsdata – t.ex. status för leder, dricksvatten och toaletter</li>"
            "<li>🧑‍🤝‍🧑🗂️ Projektytor och öppna backloggar för interoperabilitet</li>"
            "<li>🛰️ Ett gemensamt API för felanmälan och platsobservationer</li>"
            "<li>🖼️ Länkning till bilddatabaser med fria bilder och länkade data</li>"
            "<li>🔥 Tydlig ändringshantering och versionskontroll av dataposter</li>"
            "<li>🌍 Strukturerad information om 'opening hours' – idag svårt att veta vad som är öppet</li>"
            "<li>🛰️ Koppling mellan Stockholm Archipelago Trail, Skärgårdsstiftelsen och nationella datakällor</li>"
            "<li>🧩 Persistenta identifierare (URI:er) för platser och resurser – annars förloras spårbarhet "
            "se <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' target='_blank'>issue 171</a></li>"
            "</ul>"
            "Ett sådant ekosystem skulle göra det möjligt för Visit Sweden och andra aktörer att bygga verkligt datadrivna tjänster – där kartan blir en digital tvilling av verkligheten. <br />"
            "Se även <a href='https://litteraturbanken.se/litteraturkartan/' target=_blank>Litteraturkartan</a> som tyvärr inte täcker Stockholms skärgård...",
    "desc_is_html": True,
    "issues": [
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/300"
    ],
    "buzzwords": ["Digital tvilling", "Open data", "APIFirst", "Linked data", "Interoperabilitet"],
    "stakeholders": ["Vandrare", "Visit Sweden", "Skärgårdsstiftelsen"],
    "related_videos": [
  {
    "title": "Datarevolutionen bakom kartor",
    "url": "https://youtu.be/pvGUv8Bf0Go",
    "lang": "🇸🇪 Svenska",
    "comment": "Visar hur kultur och natur förenas genom öppna data – med Saltkråkan och SAT som exempel på datadriven turism (#300)."
  },
  {
    "title": "Stockholm Archipelago Trail – the Hikers Challenge – The Power of the Crowd",
    "url": "https://youtu.be/4CkxYAfV_wg",
    "lang": "🇬🇧 Engelska",
    "comment": "Betonar hur communityn (Crowd) och öppna plattformar som OSM/Wikidata kan bygga kvalitet och liv i skärgårdens data."
  },
  {
    "title": "Skapandet av den moderna leden – Stockholm Archipelago Trail",
    "url": "https://youtu.be/eB3MzlTqo9Q",
    "lang": "🇸🇪 Svenska",
    "comment": "Beskriver vägen mot en modern led med hög digital mognad (nivå 4–5) och realtidskoppling till data."
  },
  {
    "title": "Stockholm Archipelago Trail – Living Digital Twin",
    "url": "https://youtu.be/JaD7mYzlma8",
    "lang": "🇸🇪 Svenska",
    "comment": "Fokuserar på utvecklingen av en levande digital tvilling för SAT – grunden för ett datadrivet ekosystem i skärgården (#200)."
  },
  {
    "title": "Stockholm Trail’s Digital Twin",
    "url": "https://youtu.be/uGPEOp3B7WQ",
    "lang": "🇬🇧 Engelska",
    "comment": "Den engelska versionen av filmen om SAT:s digitala tvilling, med fokus på smart turism och länkad öppen data."
  },
  {
    "title": "Vilken kartdata ska vi skapa?",
    "url": "https://youtu.be/-C86emwPTjs",
    "lang": "🇸🇪 Svenska",
    "comment": "Kick-off som ringar in vilken kartdata SAT ska skapa och underhålla – OSM-relationer, stödskikt och FAIR-data (#301)."
  },
  {
    "title": "Smart Tourism – Beyond the Brochure",
    "url": "https://youtu.be/35zAwMobm3o",
    "lang": "🇬🇧 Engelska",
    "comment": "Visar hur smart turism bygger på öppna API:er och realtidsinformation – inte statiska broschyrer (#282)."
  },
  {
    "title": "Från policy till stig",
    "url": "https://youtu.be/BOUya4P1UGQ",
    "lang": "🇸🇪 Svenska",
    "comment": "Illustrerar hela kedjan från rådata till upplevelse – hur öppna data och AI formar framtidens friluftsturism (#292)."
  },
  {
    "title": "Make Hiking Data Shine",
    "url": "https://youtu.be/VH1reSTAMwY",
    "lang": "🇬🇧 Engelska",
    "comment": "Framtidsvisionen där Europas leder kopplas ihop genom FAIR och länkad öppen data – samma mål som SAT (#279)."
  },
  {
    "title": "CC 0  Nyckeln till datadriven turism",
    "url": "https://youtu.be/VH1reSTAMwY",
    "lang": "🇸🇪 Svenska",
    "comment": "Hur kan data förändra din semester"
  }

]

},
    {
        "id": "ECHOSYSTEM",
        "title": "🎥 Video – Stockholms Archipelago Trails digitala spöke",
        "url": "https://youtu.be/5atWdmH1StA",
        "img": "./thumbs/SAT_led_small.jpg",
        "desc": (
            "<p>📹 <b>Stockholms Archipelago Trails digitala spöke</b> visar hur ett digitalt ekosystem för svensk friluftsturism "
            "aldrig riktigt blivit verklighet – trots år av bidrag, projekt och nätverk.</p>"
            "<p>Idag saknas <b>uppföljning</b> av hur offentliga medel används, och informationen fastnar i <b>datasilos</b> "
            "och stängda plattformar som dessutom <b>spårar användarna</b>. Kommunikationen sker ofta via "
            "sociala medier utan koppling till öppna data, och få arbetar <b>datadrivet</b> eller med <b>API-först</b>-principer.</p>"
            "<p>Resultatet blir isolerade webbsidor på svenska istället för en sammanhållen digital infrastruktur "
            "som länkar samman data, myndigheter och besökare.</p>"
            "<ul>"
            "<li>🎯 Behövs: gemensam styrning, standarder och öppna API:er</li>"
            "<li>📊 Länkad data kan bli grunden för framtidens hållbara friluftsturism</li>"
            "<li>🧩 Se också: <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/245' target='_blank'>Issue #245</a> – Stockholms Archipelago Trails digitala spöke</li>"
            "</ul>"
        ),
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/245",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/234",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/232",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149",
        ],
        "buzzwords": [
            "AI","Ekosystem",
            "Smart tourism",
            "Open data",
            "Datadrivet",
            "Linked data"
        ],
        "stakeholders": [
            "Visit Sweden",
            "Naturvårdsverket",
            "Tillväxtverket",
            "Länsstyrelser"
        ],"related_videos": [
          {
            "title": "Stockholms Archipelago Trails digitala spöke",
            "url": "https://youtu.be/5atWdmH1StA",
            "lang": "🇸🇪 Svenska",
            "comment": "Huvudfilmen – granskar hur brist på öppna data och uppföljning skapat ett 'digitalt spöke' trots stora offentliga investeringar."
          },
          {
            "title": "CC 0  Nyckeln till datadriven turism",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇸🇪 Svenska",
            "comment": "Hur kan data förändra din semester"
          },
          {
            "title": "Vacker led, digital fälla: Är detta turismens framtid?",
            "url": "https://youtu.be/N9LI_8Zy-to",
            "lang": "🇸🇪 Svenska",
            "comment": "Ställer frågan om leden blivit en digital fälla – vacker på ytan men utan öppen data och spårbarhet (#261)."
          },
          {
            "title": "Stockholm Archipelago Trail Path or Trap – English version",
            "url": "https://youtu.be/qdIDF2SFjJE",
            "lang": "🇬🇧 Engelska",
            "comment": "Engelska versionen av samma tema: bristen på API:er och öppna data riskerar att göra leden till en digital återvändsgränd."
          },
          {
            "title": "Vackra leder, digitala fällor",
            "url": "https://youtu.be/ybWZkL8Mhpk",
            "lang": "🇸🇪 Svenska",
            "comment": "Utvecklar den kritiska diskussionen kring bristande digital infrastruktur i offentligt finansierade ledprojekt (#261)."
          },
          {
            "title": "Ett brutet löfte om öppna data",
            "url": "https://youtu.be/3oTatm6kiw4",
            "lang": "🇸🇪 Svenska",
            "comment": "Kritiserar att SAT-projektet inte levererat öppna API:er eller transparens trots tydliga löften – 'löftet om öppna data' (#245/#261)."
          },
          {
            "title": "Offentliga leder, öppen data",
            "url": "https://youtu.be/F-Te57TCKzU",
            "lang": "🇸🇪 Svenska",
            "comment": "Lyfter vikten av öppna data inom friluftsliv – hur API:er och licenser är avgörande för innovation (#262)."
          },
          {
            "title": "Gör friluftsäventyr FAIR – Stockholm Archipelago Trail",
            "url": "https://youtu.be/wJwttzo9Ao4",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur SAT kan göras FAIR (Findable, Accessible, Interoperable, Reusable) och bli en datadriven modell för andra leder (#263)."
          },
          {   
            "title": "Från Spöken till AI-guider - Generativ AI och AI agenter",
            "url": "https://youtu.be/PvIuhGHPNKU",
            "lang": "🇸🇪 Svenska",
            "comment": "AI innebär AI agenter som planerar min resa."
          },
          {
            "title": "AI i turismens framtid",
            "url": "https://youtu.be/qwY9Gc_SfMU",
            "lang": "🇸🇪 Svenska",
            "comment": "Reselandskapet i förändring. Dagens Digitala spöken är en skakig grund."
          },
          {
            "title": "AI, Resor & Osynlig Data",
            "url": "https://youtu.be/GxKw1OSUZVM",
            "lang": "🇸🇪 Svenska",
            "comment": "Från PDF till API och arbeta datadrivet och skapa digitala tvillingar."
          },
          {
            "title": "Från PDF till API – Ledens framtid",
            "url": "https://youtu.be/cGzB2ufkVeY",
            "lang": "🇸🇪 Svenska",
            "comment": "Illustrerar övergången från PDF-rapporter till öppna API:er – en nödvändighet för att undvika framtida 'digitala spöken'."
          },
          {
            "title": "Den saknade digitala visionen",
            "url": "https://youtu.be/QuK-GscJCXs",
            "lang": "🇸🇪 Svenska",
            "comment": "Fokuserar på kultur och styrning: varför Sverige saknar en samlad digital vision för öppna data och friluftsliv (#268/#270)."
          },
          {
            "title": "Den förlorade datan",
            "url": "https://youtu.be/BssSOKhaLIc",
            "lang": "🇸🇪 Svenska",
            "comment": "Beskriver de samhällsekonomiska konsekvenserna av slutna data – förlorad innovation, dubbla kostnader och låsta system (#278)."
         }
        ]
},
    {
        "id": "AI-1",
        "title": "🎥 Video – AI & Framtidens Resande",
        "url": "https://youtu.be/XtI5iqxfXWw",
        "img": "./thumbs/SAT_336_AI_400x225.jpg",
        "desc": (
            "<p>📹 <b>AI</b> framtidens resande går från visa till svara.</p>"
            "<p>Idag skapas datasilos AI behöver maskinläsbardata, öppen data</p>"
            "<ul>"
            "<li>🎯 Behövs: gemensam styrning, standarder, FAIRDATA och öppna API:er</li>"
            "<li>📊 Länkad data kan bli grunden för framtidens hållbara friluftsturism</li>"
            "<li>🧩 Se också: <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/336' target='_blank'>Issue #336</a> – AI & Framtidens Resande</li>"
            "</ul>"
        ),
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/245",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/234",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/232",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149",
        ],
        "buzzwords": [
            "AI","Ekosystem",
            "Smart tourism",
            "Open data",
            "Datadrivet",
            "Linked data"
        ],
        "stakeholders": [
            "Visit Sweden",
            "Naturvårdsverket",
            "Tillväxtverket",
            "Länsstyrelser"
        ],"related_videos": [
                      {
            "title": "Stockholms Archipelago Trails digitala spöke",
            "url": "https://youtu.be/5atWdmH1StA",
            "lang": "🇸🇪 Svenska",
            "comment": "Huvudfilmen – granskar hur brist på öppna data och uppföljning skapat ett 'digitalt spöke' trots stora offentliga investeringar."
          },
          {
            "title": "Från Spöken till AI-guider - Generativ AI och AI agenter",
            "url": "https://youtu.be/PvIuhGHPNKU",
            "lang": "🇸🇪 Svenska",
            "comment": "AI innebär AI agenter som planerar min resa."
          },
          {
            "title": "AI i turismens framtid",
            "url": "https://youtu.be/qwY9Gc_SfMU",
            "lang": "🇸🇪 Svenska",
            "comment": "Reselandskapet i förändring. Dagens Digitala spöken är en skakig grund."
          },
          {
            "title": "AI, Resor & Osynlig Data",
            "url": "https://youtu.be/GxKw1OSUZVM",
            "lang": "🇸🇪 Svenska",
            "comment": "Från PDF till API och arbeta datadrivet och skapa digitala tvillingar."
          },
        
        ]
    },
    {
        "id": "DIGITALTMUSEUM-1",
        "title": "Digitaltmuseum: Historiska bilder",
         "url": "https://wikimap.toolforge.org/?cat=SAT_Digitalt_Museum&subcats=true&subcatdepth=3&cluster=false",
        "img": "./thumbs/SAT307_Digitaltmuseum_400x225.jpg",
        "desc": "📜 Utforska den historiska kopplingen till leden och skärgården. "
        "Tyvärr finns det inga välstrukturerade arkiv med kuraterad data. "
        "Min upplevelse är att många arkiv och institutioner fortfarande saknar digital mognad "
        "och ett datadrivet arbetssätt – något som påminner mycket om situationen inom "
        "friluftslivet.",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/307",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/308"
        ],
        "buzzwords": ["Smart tourism"],
        "stakeholders": ["Visitor" ],
        "related_videos": [
          {
            "title": "Från data till kulturäventur",
            "url": "https://youtu.be/Gm_G59ViKHo",
            "lang": "🇸🇪 Svenska",
            "comment": "Möjligheterna med öppna data i olika former."
          },
          {
            "title": "FAIR Governance – att leda med data",
            "url": "https://youtu.be/8OXiiDjiOYo",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur styrning och ansvar måste bygga på FAIR-principerna (Findable, Accessible, Interoperable, Reusable)."
          },
          {
            "title": "Den saknade digitala visionen",
            "url": "https://youtu.be/QuK-GscJCXs",
            "lang": "🇸🇪 Svenska",
            "comment": "En djupare analys av hur projekt ofta fokuserar på kommunikation snarare än på öppen, maskinläsbar data."
          },
          {
            "title": "CC 0  Nyckeln till datadriven turism",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇸🇪 Svenska",
            "comment": "Hur kan data förändra din semester"
          },]
   },
  
    {
        "id": "ECHOSYSTEM-1",
        "title": "Infografik: Så här gör vi idag vs. så här borde vi göra",
         "url": "./thumbs/SATInfogrfaph1.png",
        "img": "./thumbs/SATInfogrfaph1_thumb2_small.jpg",
        "desc": "📜 Idag saknas uppföljning av hur beviljade bidrag används. "
        "Informationen hamnar ofta i datasilos, där verktyg som Google Maps dessutom spårar " 
        "användarna. Något sammanhållet ekosystem finns inte – "
        "istället sker kommunikationen mest genom glada tillrop på Instagram. "
        "Ingen arbetar datadrivet; resultatet blir ofta enbart texter på svenska på en webbsida. "
        "Myndighetsnätverk "
        "bildas och pågår i flera år utan att det leder till ett mer datadrivet arbetssätt "
        "– det känns som att en blind leder en blind. "
        "Idag saknas även uppföljning av hur beviljade bidrag används vilket uppmuntrar till lycksökare... "
        "<ul><li><a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/234' target=_blank>Issue 234</a>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/234",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/232",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149",
        ],
        "buzzwords": ["AI","Ekosystem","Smart tourism"],
        "stakeholders": ["Visit Sweden" ],
        "related_videos": [
          {
            "title": "Stockholms Archipelago Trails digitala spöke",
            "url": "https://youtu.be/5atWdmH1StA",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur offentligt finansierade initiativ för friluftsturism havererar digitalt när man saknar API:er, öppna data och styrning."
          },
          {
            "title": "Från PDF till API – Ledens framtid",
            "url": "https://youtu.be/cGzB2ufkVeY",
            "lang": "🇸🇪 Svenska",
            "comment": "Illustrerar skillnaden mellan dagens pappersbaserade leveranser och framtidens API-först-ekosystem."
          },{
            "title": "Från Spöken till AI-guider - Generativ AI och AI agenter",
            "url": "https://youtu.be/PvIuhGHPNKU",
            "lang": "🇸🇪 Svenska",
            "comment": "AI innebär AI agenter som planerar min resa."
          },
          {
            "title": "AI i turismens framtid",
            "url": "https://youtu.be/qwY9Gc_SfMU",
            "lang": "🇸🇪 Svenska",
            "comment": "Reselandskapet i förändring. Dagens Digitala spöken är en skakig grund."
          },
          {
            "title": "AI, Resor & Osynlig Data",
            "url": "https://youtu.be/GxKw1OSUZVM",
            "lang": "🇸🇪 Svenska",
            "comment": "Från PDF till API och arbeta datadrivet och skapa digitala tvillingar."
          },

          {
            "title": "The Missing Digital Vision",
            "url": "https://youtu.be/QuK-GscJCXs",
            "lang": "🇸🇪 Svenska",
            "comment": "Pekar ut att Sverige saknar en tydlig digital vision för öppna data och datadriven styrning – grunden till dagens fragmentering."
          },
          {
            "title": "FAIR Governance – att leda med data",
            "url": "https://youtu.be/8OXiiDjiOYo",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur styrning och ansvar måste bygga på FAIR-principerna (Findable, Accessible, Interoperable, Reusable)."
          },
          {
            "title": "Den saknade digitala visionen",
            "url": "https://youtu.be/QuK-GscJCXs",
            "lang": "🇸🇪 Svenska",
            "comment": "En djupare analys av hur projekt ofta fokuserar på kommunikation snarare än på öppen, maskinläsbar data."
          },
          {
            "title": "SAT: Vacker led, digital fälla?",
            "url": "https://youtu.be/N9LI_8Zy-to",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar konkreta exempel på hur samma misstag upprepas: kommunikationsprojekt utan datainfrastruktur (#261)."
          },
          {
            "title": "Offentliga leder, öppen data",
            "url": "https://youtu.be/F-Te57TCKzU",
            "lang": "🇸🇪 Svenska",
            "comment": "Lyfter behovet av att offentlig data faktiskt publiceras som öppna API:er i stället för PDF:er och broschyrer (#262)."
          },
          {
            "title": "Från policy till stig",
            "url": "https://youtu.be/BOUya4P1UGQ?si=1cAGq6OR82BKbGDZ",
            "lang": "🇸🇪 Svenska",
            "comment": "Följer kedjan från EU-policy till praktisk tillämpning i naturvården – och varför implementeringen ofta stannar vid text."
          },
          {
            "title": "CC 0  Nyckeln till datadriven turism",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇸🇪 Svenska",
            "comment": "Hur kan data förändra din semester"
          },
          {
            "title": "Från data till destination",
            "url": "https://youtu.be/BOUya4P1UGQ",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur öppen data och AI kan skapa konkreta verktyg för besöksnäringen om man bygger rätt från början (#292)."
          },
          {
            "title": "Data First – Not Documents",
            "url": "https://youtu.be/GSW9q9u1O2w",
            "lang": "🇸🇪/🇬🇧 Bilingual",
            "comment": "Ett manifest i kortformat: Digitalisering handlar inte om att skriva dokument – utan om att länka data."
          }
]

    },      {
        "id": "ECHOSYSTEM-12",
        "title": "Öppna data motorn",
         "url": "./thumbs/OppnaDataMotorn.png",
        "img": "./thumbs/OppnaDataMotorn.png",
        "desc": "📜 Öppna data ekosystem för friluftslivet "
        "Beskriver hur Stockholm Archipelago Trail dokumenterats som öppna data på öppna plattformar "
        "data som alla kan använda och bidraga till och som är FAIRDATA och 5-star data",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/313",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149",
        ],
        "buzzwords": ["AI","Ekosystem"],
        "related_videos": [
          {
            "title": "Stockholms Archipelago Trails digitala spöke",
            "url": "https://youtu.be/5atWdmH1StA",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur offentligt finansierade initiativ för friluftsturism havererar digitalt när man saknar API:er, öppna data och styrning."
          },
          {
            "title": "FAIR Governance – att leda med data",
            "url": "https://youtu.be/8OXiiDjiOYo",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur styrning och ansvar måste bygga på FAIR-principerna (Findable, Accessible, Interoperable, Reusable)."
          },
          {
            "title": "Den saknade digitala visionen",
            "url": "https://youtu.be/QuK-GscJCXs",
            "lang": "🇸🇪 Svenska",
            "comment": "En djupare analys av hur projekt ofta fokuserar på kommunikation snarare än på öppen, maskinläsbar data."
          },
          {
            "title": "SAT: Vacker led, digital fälla?",
            "url": "https://youtu.be/N9LI_8Zy-to",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar konkreta exempel på hur samma misstag upprepas: kommunikationsprojekt utan datainfrastruktur (#261)."
          },
          {
            "title": "Offentliga leder, öppen data",
            "url": "https://youtu.be/F-Te57TCKzU",
            "lang": "🇸🇪 Svenska",
            "comment": "Lyfter behovet av att offentlig data faktiskt publiceras som öppna API:er i stället för PDF:er och broschyrer (#262)."
          },
          {
            "title": "CC 0  Nyckeln till datadriven turism",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇸🇪 Svenska",
            "comment": "Hur kan data förändra din semester"
          },
          {
            "title": "Data First – Not Documents",
            "url": "https://youtu.be/GSW9q9u1O2w",
            "lang": "🇸🇪/🇬🇧 Bilingual",
            "comment": "Ett manifest i kortformat: Digitalisering handlar inte om att skriva dokument – utan om att länka data."
          }, {
            "title": "Från Spöken till AI-guider - Generativ AI och AI agenter",
            "url": "https://youtu.be/PvIuhGHPNKU",
            "lang": "🇸🇪 Svenska",
            "comment": "AI innebär AI agenter som planerar min resa."
          },
          {
            "title": "AI i turismens framtid",
            "url": "https://youtu.be/qwY9Gc_SfMU",
            "lang": "🇸🇪 Svenska",
            "comment": "Reselandskapet i förändring. Dagens Digitala spöken är en skakig grund."
          },
          {
            "title": "AI, Resor & Osynlig Data",
            "url": "https://youtu.be/GxKw1OSUZVM",
            "lang": "🇸🇪 Svenska",
            "comment": "Från PDF till API och arbeta datadrivet och skapa digitala tvillingar."
          },

]    },       {
        "id": "AI-2",
        "title": "🧩 AI-Powered Hiking Smart Trail Guide data",
        "url": "./thumbs/AI-Powered_Hiking_Smart_Trail_Guide.png",
        "img": "./thumbs/AI-Powered_Hiking_Smart_Trail_Guide.png",
        "desc": (
            "<p>🧩 <b>AI</b> framtidens behöver bra data.</p>"
            "<p>Idag skapas datasilos AI behöver maskinläsbardata, öppen data</p>"
            "<ul>"
            "<li>🎯 Behövs: gemensam styrning, standarder, FAIRDATA och öppna API:er</li>"
            "<li>📊 Länkad data kan bli grunden för framtidens hållbara friluftsturism</li>"
            "<li>🧩 Se också: <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/336' target='_blank'>Issue #336</a> – AI & Framtidens Resande</li>"
            "</ul>"
        ),
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/349",
        ],
        "buzzwords": [
            "AI","Ekosystem",
            "Smart tourism",
            "Open data",
            "Datadrivet",
            "Linked data"
        ],
        "stakeholders": [
            "Visit Sweden",
            "Naturvårdsverket",
            "Tillväxtverket",
            "Länsstyrelser"
        ],"related_videos": [
                      {
            "title": "Stockholms Archipelago Trails digitala spöke",
            "url": "https://youtu.be/5atWdmH1StA",
            "lang": "🇸🇪 Svenska",
            "comment": "Huvudfilmen – granskar hur brist på öppna data och uppföljning skapat ett 'digitalt spöke' trots stora offentliga investeringar."
          },
          {
            "title": "Från Spöken till AI-guider - Generativ AI och AI agenter",
            "url": "https://youtu.be/PvIuhGHPNKU",
            "lang": "🇸🇪 Svenska",
            "comment": "AI innebär AI agenter som planerar min resa."
          },
          {
            "title": "AI i turismens framtid",
            "url": "https://youtu.be/qwY9Gc_SfMU",
            "lang": "🇸🇪 Svenska",
            "comment": "Reselandskapet i förändring. Dagens Digitala spöken är en skakig grund."
          },
          {
            "title": "AI, Resor & Osynlig Data",
            "url": "https://youtu.be/GxKw1OSUZVM",
            "lang": "🇸🇪 Svenska",
            "comment": "Från PDF till API och arbeta datadrivet och skapa digitala tvillingar."
          },
        
        ]
    },   {
        "id": "ECHOSYSTEM-13",
        "title": "Europas Geospatiala Transformation",
         "url": "./Europas_Geospatiala_Transformation_1.pdf",
        "img": "./thumbs/Europas_Geospatiala_Transformation.jpg",
        "desc": "📜 Öppna data ekosystem för friluftslivet börjar skapas på Europa nivå var finns Sverige? "
        "Data som alla kan använda och bidraga till och som är FAIRDATA och 5-star data",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/313",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149",
        ],
        "buzzwords": ["Ekosystem"],
        "related_videos": [
          {
            "title": "Stockholms Archipelago Trails digitala spöke",
            "url": "https://youtu.be/5atWdmH1StA",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur offentligt finansierade initiativ för friluftsturism havererar digitalt när man saknar API:er, öppna data och styrning."
          },
          {
            "title": "FAIR Governance – att leda med data",
            "url": "https://youtu.be/8OXiiDjiOYo",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur styrning och ansvar måste bygga på FAIR-principerna (Findable, Accessible, Interoperable, Reusable)."
          },
          {
            "title": "Den saknade digitala visionen",
            "url": "https://youtu.be/QuK-GscJCXs",
            "lang": "🇸🇪 Svenska",
            "comment": "En djupare analys av hur projekt ofta fokuserar på kommunikation snarare än på öppen, maskinläsbar data."
          },
          {
            "title": "SAT: Vacker led, digital fälla?",
            "url": "https://youtu.be/N9LI_8Zy-to",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar konkreta exempel på hur samma misstag upprepas: kommunikationsprojekt utan datainfrastruktur (#261)."
          },
          {
            "title": "Offentliga leder, öppen data",
            "url": "https://youtu.be/F-Te57TCKzU",
            "lang": "🇸🇪 Svenska",
            "comment": "Lyfter behovet av att offentlig data faktiskt publiceras som öppna API:er i stället för PDF:er och broschyrer (#262)."
          },
          {
            "title": "CC 0  Nyckeln till datadriven turism",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇸🇪 Svenska",
            "comment": "Hur kan data förändra din semester"
          },
          {
            "title": "Data First – Not Documents",
            "url": "https://youtu.be/GSW9q9u1O2w",
            "lang": "🇸🇪/🇬🇧 Bilingual",
            "comment": "Ett manifest i kortformat: Digitalisering handlar inte om att skriva dokument – utan om att länka data."
          }
]

    },  
  {
  "id": "ECHOSYSTEM-2",
  "title": "Öppen friluftsdata 2030",
  "url": "https://docs.google.com/presentation/d/e/2PACX-1vS3hycGE4udju6IoA8_oDOqSTLxNB2x6j_TjWTlMefKdbi4NLx94on74dlxLpZcnw/pub?start=true&loop=true&delayms=3000&slide=id.p11",
  "img": "./thumbs/SAT_2030_400x225.jpg",
  "desc": "<p>📜 Idag: silos, slutna plattformar, ingen samlad styrning.</p><p>🌐 2030: öppet, API-först ekosystem som kopplar ihop data, aktörer och användare.</p><ul><li>Standarder + governance</li><li>Datadriven kommunikation – inte bara PDF:er & inlägg</li></ul>",
  "desc_is_html": True,
  "issues": [
    "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/239"
  ],
  "buzzwords": ["AI","APIFirst", "Ekosystem", "Smart tourism"],
  "stakeholders": ["Naturvårdsverket"],
  "related_videos": [
        {   
            "title": "Design för alla Smart turism",
            "url": "https://youtu.be/j-yRPRaTZHU",
            "lang": "sv",
            "comment": "Video som illustrerar konceptet 'Design för alla' och hur smart turism kan bli inkluderande och datadriven. Koppling till tillgänglighet, digital mognad och delad infrastruktur.",        
      },

      {
        "title": "Smart Tourism Beyond the Brochure",
        "url": "https://youtu.be/35zAwMobm3o",
        "lang": "🇬🇧 English",
        "comment": "Visar den europeiska visionen: öppna, sammankopplade vandringsleder via Wikidata och OSM – ett gemensamt FAIR-nätverk 2030."
      },
      {
        "title": "Sweden’s Digital Awakening",
        "url": "https://youtu.be/LG9yA7TTmOA",
        "lang": "🇬🇧 English",
        "comment": "En väckarklocka för svensk offentlig sektor: vi ligger efter i öppna data och API-först-tänkande."
      },
      {
        "title": "FAIR Governance – att leda med data",
        "url": "https://youtu.be/8OXiiDjiOYo",
        "lang": "🇸🇪 Svenska",
        "comment": "Visar hur styrning enligt FAIR-principerna är avgörande för hållbar digital förvaltning inom natur och turism."
      },
      {
        "title": "Offentlig datainfrastruktur",
        "url": "https://youtu.be/qs-WXxFZGH4",
        "lang": "🇸🇪 Svenska",
        "comment": "Förklarar hur INSPIRE-direktivet kan användas som en brygga mellan naturdata och turismdata."
      },
      {
        "title": "Sveriges Digitala Blindfläck",
        "url": "https://youtu.be/QzXfOy7-xJM",
        "lang": "🇸🇪 Svenska",
        "comment": "Visar hur andra EU-länder implementerar Open Data-direktivet praktiskt, med API:er och governance-struktur."
      },
      {
        "title": "Europas digitala ritning",
        "url": "https://youtu.be/OQRM1c9vh7Q",
        "lang": "🇸🇪 Svenska",
        "comment": "Sätter in friluftsdata i det större perspektivet av EU:s digitala strategi och interoperabilitet 2030."
      },
      {
        "title": "Smart Tourism – Beyond the Brochure",
        "url": "https://youtu.be/35zAwMobm3o",
        "lang": "🇬🇧 English",
        "comment": "Visar hur öppna API:er och realtidsinformation skapar datadriven turism – istället för PDF-broschyrer."
      },
      {
        "title": "Datarevolutionen bakom kartor",
        "url": "https://youtu.be/pvGUv8Bf0Go",
        "lang": "🇸🇪 Svenska",
        "comment": "Knyter samman kultur, natur och turism via öppen länkad data – ett exempel på framtidens datadrivna ekosystem."
      }
  ]

},
{
    "id": "ECHOSYSTEM-3",
    "title": "👉 📜 User cases SAT",
    "url": "./SAT_ALL_IN_ONE_142_3_dashboard_latest.html#UC-1",
    "img": "./thumbs/SAT_UserCases_400x225.jpg",
    "desc": "The first step in creating high-quality data models is to identify and describe the stakeholders (<i>intressenter</i>) — "
        "in <a href='https://en.wikipedia.org/wiki/Unified_Modeling_Language' target=_blank>UML terms</a>, this means defining the "
        "actors and their roles. A key lesson from the first year of the <a href='https://stockholmarchipelagotrail.com' target=_blank>SAT project</a> "
        "is that few people understand the importance of this step, and data quality is often not treated as a priority. "
        "This challenge is not unique to SAT — it reflects the broader immaturity of outdoor data (<i>Friluftsdata</i>). "
        "Developing clear <a href='https://www.scrum.org/resources/blog/user-story-format' target=_blank>user stories</a> could be a good starting point.",
    "desc_is_html": True,
    "issues": [
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/229",
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/232",
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/239"
    ],
    "buzzwords": ["AI","Ekosystem", "Smart tourism"],
    "stakeholders": ["Vandrare", "Visit Sweden"],
    "related_videos": [
          {
            "title": "Från backlog till verklighet",
            "url": "https://youtu.be/1fcyZA_okus",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur agila metoder och öppna GitHub-backlogs kan användas för att utveckla användarfall och skapa transparens."
          },
          {
            "title": "API first Sweden",
            "url": "https://youtu.be/qN0Cbl8Hsv4",
            "lang": "🇸🇪 Svenska",
            "comment": "Förklarar varför API-först-tänkandet är nödvändigt för att bygga hållbara användarflöden i digitala ekosystem."
          },
          {
            "title": "Datarevolutionen bakom kartor",
            "url": "https://youtu.be/pvGUv8Bf0Go",
            "lang": "🇸🇪 Svenska",
            "comment": "Knyter ihop natur, kultur och användarupplevelse via öppen, länkad data — precis vad User Cases i SAT handlar om."
          },
          {
            "title": "The Hikers Challenge – The Power of the Crowd",
            "url": "https://youtu.be/4CkxYAfV_wg",
            "lang": "🇬🇧 English",
            "comment": "Visar hur användare och community kan bidra med datainsamling och kvalitetssäkring av vandringsleder."
          },
          {
            "title": "The FAIR Trail Network",
            "url": "https://youtu.be/35zAwMobm3o",
            "lang": "🇬🇧 English",
            "comment": "Sätter användarfallen i ett europeiskt perspektiv – öppna, interoperabla leder kopplade till aktörer och dataflöden."
          },
          {
            "title": "Hiking Maturity Model",
            "url": "https://youtu.be/SzkKbzsfyRU",
            "lang": "🇸🇪 Svenska",
            "comment": "Ramverket som definierar digital mognad för leder – från ostrukturerade PDF:er till smarta användarfall och realtidsdata."
          },
          {
            "title": "Agile Linked Data – Sustainable Trails",
            "url": "https://youtu.be/FW6MDhoCEKk",
            "lang": "🇬🇧 English",
            "comment": "Förklarar hur agila metoder och länkad data samverkar för att skapa flexibla och hållbara användarflöden."
          },
          {
            "title": "Den smarta leden",
            "url": "https://youtu.be/mKvdCT2wF94",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur framtidens vandringsleder bygger på smart data och användarorienterade scenarier med realtidsinformation."
          }
        ]
},
 
    {
        "id": "ECHOSYSTEM-4",
        "title": "👉 📜 Machine-translated regulations for nature reserves along SAT",
        "url": "./SAT_195_MachineTranslate_latest.html",
        "img": "./thumbs/SAT195_MachineTranslation_400x225.jpg",
        "desc": "📜 One key stakeholder for SAT is visitors from abroad, who need to understand "
        "the nature reserve regulations that are currently published only in Swedish. "
        "This map use Google Translate for the most common languages spoken in Sweden and by tourist"
        "<br /><br /><a href='https://youtu.be/4MmD1EctW2Y' target=_blank>▶️ video</a>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/195",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/78"
        ],
        "buzzwords": ["AI","Ekosystem", "Flerspråkighet","Smart tourism"],
        "stakeholders": ["Naturvårdsverket", "Non-Swedish speaking hikers","Skärgårdsstiftelsen", "Vandrare","Visit Sweden" ],
        "related_videos": [
          {
            "title": "Den universella leden – Stockholm Archipelago Trail",
            "url": "https://youtu.be/tjfSc8FdKjo",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur Wikidata används för att skapa en flerspråkig och inkluderande digital infrastruktur för SAT."
          },
          {
            "title": "Szlak Mówiący Twoim Językiem",
            "url": "https://youtu.be/fyjOYjfXt6c",
            "lang": "🇵🇱 Polska",
            "comment": "Ett exempel på hur språkoberoende metadata gör leder tillgängliga även för utländska besökare – på polska och andra språk."
          },
          {
            "title": "數據破解斯德哥爾摩群島步道",
            "url": "https://youtu.be/HqQoNkjwdC8",
            "lang": "🇹🇼 Kinesiska (Traditionell)",
            "comment": "Beskriver hur standardiserad geodata och Wikidata kan användas för att 'avkoda' leder för kinesisktalande turister."
          },
          {
            "title": "Stockholm Archipelago Trail – the Hikers Challenge",
            "url": "https://youtu.be/4CkxYAfV_wg",
            "lang": "🇬🇧 English",
            "comment": "Belyser vikten av community-drivna initiativ för att samla in och kvalitetssäkra data som alla, oavsett språk, kan använda."
          },
          {
            "title": "Il Puzzle dell'Arcipelago",
            "url": "https://youtu.be/nHeCo3hL64o",
            "lang": "🇮🇹 Italienska",
            "comment": "Visar hur projektet binder ihop fragmenterad information till en sammanhållen flerspråkig upplevelse för turister."
          },
          {
            "title": "The FAIR Trail Network",
            "url": "https://youtu.be/35zAwMobm3o",
            "lang": "🇬🇧 English",
            "comment": "Sätter arbetet i ett europeiskt sammanhang: alla Europas leder tillgängliga via öppna, interoperabla språkneutrala data."
          },
          {
            "title": "Stockholm Archipelago Trail living Digital Twin",
            "url": "https://youtu.be/JaD7mYzlma8",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur en levande digital tvilling kan leverera realtidsöversättningar, statusdata och språkoberoende metadata."
          },
          {
            "title": "Stockholms skjulte utfordring",
            "url": "https://youtu.be/_7IiAPmiQ1I",
            "lang": "🇳🇴 Norska",
            "comment": "Belyser de nordiska språkbarriärerna och hur Wikidata löser dem genom gemensam semantisk modellering."
          }
        ]

    },  
    {
        "id": "RAA-1",
        "title": "🖼️ RAÄ Bebyggelseregistret - SAT",
        "url": "./SAT_BBR_latest.html",
        "img": "./thumbs/SAT_BBR_220_400x225.jpg",
        "desc": "🖼️ Example of using BBR to discover interesting points of interest (POIs) along the SAT trail."
        "<br /><br/>* <a target=_blank href='https://www.raa.se/hitta-information/oppna-data/oppna-data-portal/'>Öppna data</a>"
        "<br />* <a target=_blank href='https://www.raa.se/hitta-information/bebyggelseregistret/'>Bebyggelseregistret (BBR)</a>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/220"
        ],
        "buzzwords": ["Smart tourism"],
        "stakeholders": ["Vandrare"],
        "related_videos": [
          {
            "title": "Datarevolutionen bakom kartor",
            "url": "https://youtu.be/pvGUv8Bf0Go",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur kultur och natur möts genom datadrivna verktyg – kopplingen mellan kulturarv, filmer och naturleder."
          },
          {
            "title": "Stockholm Archipelago Trail – the Hikers Challenge",
            "url": "https://youtu.be/4CkxYAfV_wg",
            "lang": "🇬🇧 English",
            "comment": "Om hur communityn kan bidra till att dokumentera kulturarv och platser längs leden via OSM och Wikidata."
          },
          {
            "title": "Smart Tourism – Beyond the Brochure",
            "url": "https://youtu.be/35zAwMobm3o",
            "lang": "🇬🇧 English",
            "comment": "Visar hur smart turism bygger på öppna data och kulturkopplingar – inte bara marknadsföring."
          },
          {
            "title": "Att digitalisera skärgården",
            "url": "https://youtu.be/NDeKbyfGmPk",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar tidigare försök att digitalisera skärgårdens kultur- och naturarv, och varför öppen data är avgörande."
          },
          {
            "title": "Bortom ledmarkeringarna",
            "url": "https://youtu.be/_oR8I80JYP0",
            "lang": "🇸🇪 Svenska",
            "comment": "Fokuserar på att berika leder med digital kulturinformation och metadata – utöver fysiska skyltar."
          },
          {
            "title": "Vilken kartdata ska vi skapa?",
            "url": "https://youtu.be/-C86emwPTjs",
            "lang": "🇸🇪 Svenska",
            "comment": "Tar upp vikten av att välja rätt kartdata, inklusive kulturarvsobjekt som stödskikt nära vandringsleder."
          },
          {
            "title": "Länkad data: Kartlägger vår värld",
            "url": "https://youtu.be/3JjOkCv5BeM",
            "lang": "🇸🇪 Svenska",
            "comment": "Förklarar hur länkad data binder samman RAÄ:s, OSM:s och Wikidatas resurser för att skapa ett rikt kultur- och naturarvsekosystem."
          }
        ]

    },   
    {
    "id": "ECHOSYSTEM-5",
    "title": "👉 Shifting from PDFs and projects to data and ecosystems — what kind of expertise do we need?",
    "url": "./KompetenserDatadrivet.png",
    "img": "./thumbs/Kompetensersombeh%C3%B6vs.png",
    "desc": "To move from scattered <b>PDF reports</b> and short-term projects to sustainable, "
    "open <b>data ecosystems</b>, we need new kinds of expertise — people who understand how information "
    "flows between platforms through <b>semantic links</b> rather than file attachments."
    "<br /><br />This shift means working <b>language-independent (språkoberoende)</b>, "
    "where every concept — a trail, a nature reserve, or a water tap — has a stable, "
    "global identifier in <a href='https://www.wikidata.org' target='_blank'>Wikidata</a> or "
    "<a href='https://www.openstreetmap.org' target='_blank'>OpenStreetMap</a>. Once connected, data "
    "can be reused across languages, apps, and countries without manual translation or duplication.<br /><br />"
    "Such <b>semantic, API-first data</b> enables <a href='https://www.go-fair.org/fair-principles/' target='_blank'>FAIR</a> principles "
    "— <i>Findable, Accessible, Interoperable, Reusable</i> — and creates the foundation for <b>smart tourism</b> and collaboration. "
    "The goal is a living ecosystem where local contributions automatically gain global visibility "
    "and meaning.<br /><br />This requires a mix of <b>open data literacy</b>, <b>semantic modeling</b>, "
    "<b>multilingual thinking</b>, and <b>community collaboration</b> — turning yellow and blue ribbons "
    "on trees into connected digital knowledge.",
    "desc_is_html": True,
    "buzzwords": ["AI","Ekosystem", "FAIR data", "Flerspråkighet", "Linked data", "Smart tourism"],
    "stakeholders": ["Data modelers", "Digital strategists", "Non-Swedish speaking hikers", "Vandrare", "Tourism developers", "Community driven"],
"related_videos": [
  {
    "title": "Från Papper till Plattform",
    "url": "https://youtu.be/sUhx2HFUCbU",
    "lang": "🇸🇪 Svenska",
    "comment": "Visar vägen från PDF-dokument till datadrivna plattformar – en konkret illustration av skiftet som ECHOSYSTEM-5 beskriver."
  },
  {
    "title": "Data First – Not Documents",
    "url": "https://youtu.be/GSW9q9u1O2w",
    "lang": "🇬🇧 English",
    "comment": "Ett kort manifest som förkroppsligar idén om att digitalisering handlar om dataflöden, inte dokument."
  },
  {
    "title": "FAIR Governance – att leda med data",
    "url": "https://youtu.be/8OXiiDjiOYo",
    "lang": "🇸🇪 Svenska",
    "comment": "Förklarar hur FAIR-principerna (Findable, Accessible, Interoperable, Reusable) ska genomsyra offentlig styrning och datadrivet ledarskap."
  },
  {
    "title": "Hiking Maturity Model",
    "url": "https://youtu.be/SzkKbzsfyRU",
    "lang": "🇸🇪 Svenska",
    "comment": "Introducerar en modell för att bedöma digital mognad inom vandringsleder – från PDF-kaos till Smart Data-ekosystem."
  },
  {
    "title": "Agile Linked Data: Sustainable Trails",
    "url": "https://youtu.be/FW6MDhoCEKk",
    "lang": "🇬🇧 English",
    "comment": "Visar hur agila metoder och länkad data kan användas för att skapa hållbara, öppna ekosystem för vandringsleder."
  },
  {
    "title": "Den saknade digitala visionen",
    "url": "https://youtu.be/QuK-GscJCXs",
    "lang": "🇸🇪 Svenska",
    "comment": "Belyser behovet av ledarskap och kompetens för att skapa en gemensam digital vision inom svensk offentlig sektor."
  },
  {
    "title": "Open Data Act in Practice",
    "url": "https://youtu.be/QzXfOy7-xJM",
    "lang": "🇸🇪 Svenska",
    "comment": "Visar hur EU:s öppna datadirektiv kräver nya kompetenser och API-först tänkande i både offentlig sektor och näringsliv."
  },
  {
    "title": "Marketing to Data Stewardship",
    "url": "https://youtu.be/--6JxYIxJoI",
    "lang": "🇬🇧 English",
    "comment": "Omvandlar marknadsföring till datastyrning – från storytelling till datadriven strategi."
  },
  {
    "title": "The FAIR Trail Network",
    "url": "https://youtu.be/35zAwMobm3o",
    "lang": "🇬🇧 English",
    "comment": "Sätter det hela i ett europeiskt sammanhang – där kompetens i FAIR och semantik blir grunden för framtidens öppna leder."
  }
]
        
},
    {
    "id": "ECHOSYSTEM-6",
    "title": "👉 Be part of the Waymarked Hiking platform",
    "url": "https://hiking.waymarkedtrails.org/#?map=4.0/52.3623/19.1855",
    "img": "./thumbs/waymarkedtrails_400x225.jpg",
    "desc": "Platforms like <a href='https://hiking.waymarkedtrails.org' target='_blank'>WaymarkedTrails</a> show how "
    "open data from <a href='https://www.openstreetmap.org' target='_blank'>OpenStreetMap</a> creates a living ecosystem for hiking. "
    "Every trail and point of interest added to OSM is instantly reused across apps, maps, and websites — "
    "turning local mapping into global visibility.<br /><br />This open, multilingual and API-first ecosystem follows "
    "the <a href='https://www.go-fair.org/fair-principles/' target='_blank'>FAIR Data</a> vision: making hiking routes"
    "<b>Findable, Accessible, Interoperable, and Reusable</b>. It connects hikers, tourism operators, and local communities "
    "in a feedback loop where better data means better experiences.<br /><br />A vision that goes beyond just photos on Instagram "
    "— transforming yellow and blue ribbons on trees into connected digital trails that everyone can find, explore, and improve.",
    "desc_is_html": True,
    "buzzwords": ["APIFirst", "FAIR data", "Ekosystem", "Linked data", "Smart tourism"],
    "stakeholders": ["Non-Swedish speaking hikers", "Vandrare", "Tourism developers"],
    "related_videos": [
            {
              "title": "The FAIR Trail Network",
              "url": "https://youtu.be/VH1reSTAMwY",
              "lang": "🇬🇧 English",
              "comment": "Explores how FAIR principles — Findable, Accessible, Interoperable, Reusable — can transform hiking data into a shared European infrastructure."
            },
            {
              "title": "Agile Linked Data: Sustainable Trails",
              "url": "https://youtu.be/FW6MDhoCEKk",
              "lang": "🇬🇧 English",
              "comment": "Shows how open, linked data makes trail information reusable across countries and platforms — exactly what Waymarked Trails demonstrates."
            },
            {
              "title": "Datarevolutionen bakom kartor",
              "url": "https://youtu.be/pvGUv8Bf0Go",
              "lang": "🇸🇪 Svenska",
              "comment": "Illustrates how Wikidata, OSM and Commons together create living, interoperable maps — the same foundation WaymarkedTrails builds on."
            },
            {
              "title": "Make Hiking Data Shine",
              "url": "https://youtu.be/VH1reSTAMwY",
              "lang": "🇬🇧 English",
              "comment": "Advocates for FAIR hiking data and visualisation — showing the direct benefits of open, reusable trail information."
            },
            {
              "title": "Smart Tourism – Beyond the Brochure",
              "url": "https://youtu.be/35zAwMobm3o",
              "lang": "🇬🇧 English",
              "comment": "Discusses how smart tourism builds on open APIs and real-time linked data, enabling platforms like Waymarked Trails to thrive."
            }
          ]        
},
    {
    "id": "ECHOSYSTEM-11",
    "title": "🗺️ Be part of OpenStreetMap",
    "url": "https://wiki.openstreetmap.org/wiki/Hiking",
    "img": "https://wiki.openstreetmap.org/w/images/thumb/c/cd/Surveyor_Sticker.svg/480px-Surveyor_Sticker.svg.png",
    "desc": "<a href='https://www.openstreetmap.org' target='_blank'>OpenStreetMap (OSM)</a> is the "
    "backbone of the global hiking and outdoor data ecosystem — powering platforms like WaymarkedTrails, "
    "Komoot, and countless local apps.<br /><br />Every path, viewpoint, and toilet mapped in OSM becomes "
    "part of a shared public dataset that anyone can reuse. "
    "This openness transforms local contributions into global visibility and collaboration.<br /><br />"
    "To connect OSM with the wider web of knowledge, we need semantic bridges — linking OSM features "
    "to <a href='https://www.wikidata.org' target='_blank'>Wikidata</a> items using "
    "<a href='https://www.w3.org/TR/skos-reference/' target='_blank'>SKOS</a> and <b>Linked Data</b> principles. "
    "This enables FAIR integration — <b>Findable, Accessible, Interoperable, Reusable</b> — "
    "so that hiking data can move seamlessly between maps, apps, and knowledge graphs.<br /><br />"
    "OSM defines how a hiking trail should be described; see the <a href='https://wiki.openstreetmap.org/wiki/Hiking' target='_blank'>OSM "
    "Hiking documentation</a> to start contributing and strengthen the ecosystem.",
    "desc_is_html": True,
    "buzzwords": ["Open data", "FAIR data", "Linked data", "Community driven"],
    "stakeholders": ["Local mappers", "Data modelers", "Hiking platforms", "Tourism developers", "Researchers"],
    "related_videos": [
          {
            "title": "The Power of the Crowd – Stockholm Archipelago Trail",
            "url": "https://youtu.be/4CkxYAfV_wg",
            "lang": "🇬🇧 English",
            "comment": "Shows how community-driven mapping through OSM and Wikidata creates global visibility for local trails and amenities."
          },
          {
            "title": "Agile Linked Data: Sustainable Trails",
            "url": "https://youtu.be/FW6MDhoCEKk",
            "lang": "🇬🇧 English",
            "comment": "Demonstrates how linked data and FAIR principles connect OSM with Wikidata and other open datasets for seamless interoperability."
          },
          {
            "title": "Make Hiking Data Shine",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇬🇧 English",
            "comment": "Highlights how OSM and Wikidata can make hiking data findable, accessible, interoperable, and reusable — FAIR by design."
          },
          {
            "title": "Datarevolutionen bakom kartor",
            "url": "https://youtu.be/pvGUv8Bf0Go",
            "lang": "🇸🇪 Svenska",
            "comment": "Explains how Wikidata, OSM and Commons form the foundation of modern open geodata ecosystems."
          },
          {
            "title": "Smart Tourism – Beyond the Brochure",
            "url": "https://youtu.be/35zAwMobm3o",
            "lang": "🇬🇧 English",
            "comment": "Shows how open mapping and semantic connections drive smarter, data-first tourism ecosystems."
          },
          {
            "title": "The FAIR Trail Network",
            "url": "https://youtu.be/35zAwMobm3o?t=70",
            "lang": "🇬🇧 English",
            "comment": "Visualizes how FAIR data integration across OSM, Wikidata and other sources enables real-time, open trail networks worldwide."
          }
        ]
},
{
    "id": "IMAGE-1",
    "title": "🖼️ Clear image licenses with free-to-use pictures",
    "url": "./237_SAT_images.html",
    "img": "./thumbs/237_SAT_images_400x225.jpg",
    "desc": (
        "🖼️ This map shows more than 5000 freely licensed images connected to the "
        "Stockholm Archipelago Trail. All photos can be safely reused by journalists, "
        "guidebook authors, and travel companies — thanks to clear Creative Commons "
        "licenses and metadata from Wikimedia Commons and Wikidata."
        "<br/><br/>It demonstrates the value of open image ecosystems where media "
        "creators and tourism stakeholders can collaborate using linked open data."
    ),
    "desc_is_html": True,
    "issues": [
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/237",
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/305"
    ],
    "buzzwords": ["AI","Open data", "Creative Commons", "Smart tourism"],
    "stakeholders": ["Community", "Journalists", "Photographers"]
},    {
        "id": "ECHOSYSTEM-8",
        "title": "👉 📜 SAT / Wikidata / OSM / Social media / Opening hours",
        "url": "./SAT_217_WIkipedia_OSM_latest.html",
        "img": "./thumbs/SAT_OSM_WD_217_400x225.jpg",
        "desc": "📜 This prototype shows how the Stockholm Archipelago Trail can be managed in a data-driven way using Linked Open Data. "
        "By connecting Wikidata, OSM, and multilingual resources, regulations and trail information can be dynamically presented not only in Swedish but also in the languages most spoken by visitors. "
        "<br/><br/>Such an approach makes it possible to link content directly to Google Maps, social media platforms, and other digital services, enabling smarter tourism experiences. "
        "<br /><br /><a href='https://youtu.be/_nbI8hkRAvA' target=_blank>▶️ video</a>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/217",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149"
        ],
        "buzzwords": ["AI","APIFirst", "Ekosystem", "Linked data","Smart tourism"],
        "stakeholders": ["Community", "Non-Swedish speaking hikers", "Vandrare"]        
    }, 
    {
    "id": "ECHOSYSTEM-9",
    "title": "👉 📜 Wikidata graf",
    "url": "https://w.wiki/EMLz",
    "img": "./thumbs/SAT_WD-graph_400x225.jpg",
    "desc": "📜 <b>Wikidata</b> fungerar som den semantiska bryggan i ekosystemet — där varje "
    "vandringsled, naturreservat och delsträcka får en unik identifierare (Q-nummer) som kan länka till OpenStreetMap, "
    "WaymarkedTrails, Wikipedia och lokala databaser.<br /><br />Detta gör informationen <b>språkagnostisk</b> "
    "— samma objekt kan visas på svenska, engelska eller tyska utan att duplicera data. "
    "Genom <b>länkade data</b> kan olika källor förstå och använda varandras information, vilket möjliggör "
    "<b>FAIR-data</b> (Findable, Accessible, Interoperable, Reusable).<br /><br />Resultatet är ett "
    "<b>smart turist- och kunskapsekosystem</b> där data inte längre är inlåst i siloer utan fritt kan "
    "flöda mellan kartor, appar och språk.<br /><br />📊 Se grafen som visar hur SAT kopplas till sina delsträckor, "
    "naturreservat och dricksvattenpunkter via Wikidata.",
    "desc_is_html": True,
    "issues": [
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/217",
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/244"
    ],
    "buzzwords": ["Ekosystem", "FAIR data", "Linked data", "Smart tourism"],
    "stakeholders": ["Community", "Non-Swedish speaking hikers", "Vandrare", "Turismaktörer"]
},

    
 {
        "id": "WATER-1",
        "title": "🚻 Toaletter nära leden",
        "url": "./Issue_132_2_toaletter_nara_stockholm_archipelago_trail_latest.html",
        "img": "./thumbs/SAT_Map_132_toilets.jpg",
        "desc": "🚻 Söker i OpenStreetMap efter toaletter längs leden och ser vilket metadata "
        "som saknas. Idag saknas realtidsinfo och kopplingar till aktuell status – trots att "
        "Skärgårdsstiftelsen fått 1 miljon Euro för digitalisering."
        "<br/><br /><b>Önskvärt vore stöd för:</b>"
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ställer Tillväxtsverket inga krav på koppling hjärtstartare <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder för hjärtstartare - länkade data</li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och har med <b>Toaletter som friluftslivsdata</b> som skall levereras"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/132",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/140",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/127",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["APIFirst","Bilddatabas","Ekosystem", "Flerspråkighet","Smart tourism", "Open data"],
        "stakeholders": ["Naturvårdsverket", "Non-Swedish speaking hikers","Skärgårdsstiftelsen", "Vandrare", ]        
    },
    {
        "id": "WATER-2",
        "title": "💧 Dricksvatten nära leden",
        "url": "https://salgo60.github.io/Stockholm_Archipelago_Trail/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_latest.html",
        "img": "./thumbs/SAT_Map_139_drinkingwater.jpg",
        "desc": "💧 Söker i OpenStreetMap efter dricksvatten längs leden. "
        "Idag saknas ett ekosystem där man digitalt kan se vad som är påslaget och testat. "
        "Öppna data från Skärgårdsstiftelsen borde ge exempel på hur skärgården beskrivs digitalt."
        "<br/><br /><b>Önskvärt vore stöd för:</b>"
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ställer Tillväxtsverket inga krav på koppling hjärtstartare <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder för hjärtstartare - länkade data</li>"
        "<li>🌍 'samma som' dvs. Linked data där vi skall kunna följa från ansökan - bidrag -leverans inte dagens pdf:er</li>"
        "<li>🛰️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och har med <b>tillgång till dricksvatten som friluftslivsdata</b>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/139",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/140",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["AI","APIFirst","Bilddatabas", "Ekosystem","Interoperabilitet", "SSOT","Open data"],
        "stakeholders": ["Naturvårdsverket", "Non-Swedish speaking hikers","Skärgårdsstiftelsen", "Vandrare", ]        
    },
    {
        "id": "VISITSWEDEN-1",
        "title": "🌍 SAT - Visit Sweden API",
        "url": "./222_visitsweden_with_trail_latest.html",
        "img": "./thumbs/SAT_VisitSweden_222_400x225.jpg",
        "desc": "🌍 Visit Sweden jobbar datadrivet hur långt har dom kommit? <br /><br />"
        "Det vi ser i <a href='https://www.facebook.com/groups/2875020699552247' target=_blank>FB gruppen</a> är behov av att enkelt "
        "hitta resa, är dricksvattnet på, är toaletter handikappanpassade, extra erbjudande, "
        "är det problem på vandringsleden... <br /><br />"
        "Idag saknas ett ekosystem där man digitalt kan se vad som är öppet. "
        "Öppna data från Skärgårdsstiftelsen skall finnas och borde ge exempel på hur skärgården beskrivs digitalt."
        "<br/><br /><b>Önskvärt vore stöd för:</b>"
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder - länkade data</li>"
        "<li>🔥 tydlig ändringshantering av poster och versionshantering</li>"
        "<li>🌍 'opening hours' idag är det enormt svårt att se vad som är öppet och var dricksvatten toaletter finns</li>"
        "<li>🛰️ saknar helt koppling Stockholm Archipelago Trail i API:et och till Skärgårdsstiftelsen</li>"
        "<li>🔥 skall saker som detta fungera måsta man ha tydliga persistenta identifierare känns tveksamt "
        "se issue  <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' target='_blanket'>issue 171</a>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/222",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/151",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["APIFirst","Bilddatabas", "Ekosystem","Interoperabilitet","Open data"],
        "stakeholders": ["Non-Swedish speaking hikers","Skärgårdsstiftelsen","Vandrare", "Visit Sweden"]        
    },  

    {
        "id": "IMAGE-2",
        "title": "🖼️ Koppling mellan led och bild – Image Quality Control",
        "url": "./SAT_ALL_IN_ONE_142_3_image_qc_latest.html",
        "img": "./thumbs/SAT_Image_Quality_Control_400x225.jpg",
        "desc": "🖼️ Hela leden är dokumenterad i OSM där varje etapp "
        "och delsträcka kopplas till bilder på Wikimedia Commons. "
        "Denna karta låter dig kvalitetssäkra både kartinfo och bildmaterial. "
        "Innehåller lager för 🚻 toaletter och 💧 dricksvatten."
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ bilddatabaser med fria bilder och länkade data</li></ll>",
        "desc_is_html": True,
        "issues": ["https://github.com/salgo60/ProjectOutdoorGyms/issues/74",
                  "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"],
        "buzzwords": ["AI","APIFirst","Bilddatabas","Ekosystem", "Interoperabilitet", "Open data"],
        "stakeholders": ["Naturvårdsverket"]        
    },
    {
        "id": "WHEELCHAIR-1",
        "title": "♿ Funktionstillgänglighet – rullstol",
        "url": "./SAT_WHEELCHAIR_073_wheelchair_latest.html",
        "img": "./thumbs/SAT_Wheelchair_thumbnail.jpg",
        "desc": "♿ Visar objekt längs leden som är taggade 'wheelchair' i OSM. "
        "Även här efterlyses bättre öppna data från Skärgårdsstiftelsen. "
        "<strong>Naturvårdsverket</strong> som alla tror är expertmyndighet ser vi har funderat sedan 2019. "
        "När till och med Myndigheten för delaktighet sitter med i Myndighetsnätverket om vandringsleder men man sedan 2019 inte lyckats "
        "ställa tydliga krav på tillgänglighet eller leverera data som beskriver detta – då är det uppenbart att det är fel laguppställning. "
        "Fortsätter det så här kommer ingenting att vara på plats ens 2035. Hur svårt kan det vara... med chatGPT 5 sekunder.."
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet<li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li></ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/73",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/172",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/174"
        ],
        "buzzwords": ["AI","Ekosystem", "Interoperabilitet", "SSOT", "Open data"],
        "stakeholders": ["Naturvårdsverket", "Vandrare", "Skärgårdsstiftelsen"]        
    },
    {
        "id": "DOGPARK-1",
        "title": "✅ SAT Dog parks",
        "url": "./SAT225_hundparker.html",
        "img": "./thumbs/SAT_225_DogParks_small.jpg",
        "desc": "✅ Visualiserar hundrastgårdar nära SAT leden.<br /><br />"
        "Status idag är att det inte finns några bra officiella källor om hundrastgårdar.<br /><br />"
        "Vad som gjordes"
        "<ul><li>data hämtades från ett privat initiativ hundlistan och Open Street Map "
        "se <a target=_blank href='https://github.com/salgo60/Dogpark_Sweden'>GITHUB Dogparks Sweden</a></li></ul>",
        "desc_is_html": True,
        "issues": ["https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/225"],
        "buzzwords": [ "Community driven", "SSOT"],
        "stakeholders": ["Community","Hundägare","Kommuner",]        
    },
    {
        "id": "OSM-1",
        "title": "✅ Att göra i OSM – Todo",
        "url": "./SAT_ALL_IN_ONE_142_3_ALL_latest.html",
        "img": "./thumbs/SAT_Audit_layers_thumbnail.jpg",
        "desc": "✅ Visualiserar saknade OSM-taggar på en karta över leden.",
        "issues": ["https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"],
        "buzzwords": ["Interoperabilitet"],
        "stakeholders": ["Community","Naturvårdsverket"]        
    },
    {
        "id": "OSM-2",
        "title": "🔍 Audit-lager i OSM",
        "url": "./SAT_ALL_IN_ONE_142_3_split_todo_latest.html",
        "img": "./thumbs/SAT_Audit_layers_thumbnail.jpg",
        "desc": "🔍 Karta som visar var OSM-taggar saknas för surface / foot / sac / trail_visibility / step_count.",
        "issues": ["https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"],
        "buzzwords": ["Interoperabilitet", "Open data"],
        "stakeholders": ["Community","Naturvårdsverket"]        
    },
    {
        "id": "OSM-3",
        "title": "🪜 Trappsteg på leden – Steps",
        "url": "./SAT_ALL_IN_ONE_142_3_steps_only_latest.html",
        "img": "./thumbs/SAT_Steps_Only_thumbnail.jpg",
        "desc": "🪜 SAT-leden har begärt ersättning för ~200 m trappor "
        "– men de är svåra att hitta. Är det detta som är överraskningen att inget levereras?",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["Bilddatabas","Interoperabilitet", "SSOT","Tillgänglighet"],
        "stakeholders": ["Tillväxtverket", "Vandrare"]        
    },
    {
        "id": "OSM-4",
        "title": "🌊 Närhet till vatten – Proximity",
        "url": "./SAT_ALL_IN_ONE_142_3_water_proximity_latest.html",
        "img": "./thumbs/SAT_Water_Proximity_800x450.jpg",
        "desc": "🌊 Vissa sträckor går långt från vattnet – på gott och ont. Kartan visar hur nära/långt från vattnet olika delar ligger, samt längsta distans från vatten.",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": [ "Interoperabilitet", "Smart tourism","Open data"],
        "stakeholders": ["Vandrare"]        
    },
    {
        "id": "WIKIPEDIA-1",
        "title": "📚 Wikipedia/Wikidata",
        "url": "./SAT_ALL_IN_ONE_142_3_Wikipedia.html",
        "img": "./thumbs/SAT_Map_Wikipedia_400x225.jpg",
        "desc": "📚 Mer än 600 Wikipedia/Wikidata-objekt har kopplats till SAT-leden. "
        "Tanken är att visa på hur <strong>digital interoperabilitet</strong> fungerar i praktiken, "
        "och hur man 2025 enkelt kan leverera flerspråkighet med vettiga ekosystem som hanterar"
        "persistenta identifierare och api:er <br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🌍 flerspråkig data</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li>"
        "<li>🛰️ Länkade data samma som jmf <a href='https://github.com/salgo60/NOSAD-POC-Wikidata/issues/13' target=_blank >hur vi jobbar med Nobelprize.org</a> och uppdaterar Wikipedia via Wikidata</li>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/22",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": [ "APIFirst", "Community driven", "Non-Swedish speaking hikers",  "Flerspråkighet", "Interoperabilitet", "Smart tourism"],
        "stakeholders": ["Community","Vandrare"]        
    },
    {
        "id": "SIGN-1",
        "title": "🪧 SAT infoskyltar",
        "url": "./182_WD_OSM_signs_latest.html",
        "img": "./thumbs/SAT_InfoSign_400x225.jpg",
        "desc": "🪧 135 turistinformationsskyltar beställdes via Tillväxtverket – endast ~80 hittade. "
        "Ingen öppen data om placering eller leveransstatus. "
        "Vi skall överraskas - ja vi är överraskade hur dumt skattepengar slösas "
        "<i>The Magic slöseri</i><br/>"
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 flerspråkig data</li>"
        "<li>♿ tydligare tillgänglighet - följ internationell klassificering</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder på alla skyltarna som har unika id:n</li>"
        "<li>⏱️ information om tid att vandra sträckan</li>"
        "<li>⚠️ status på leden med senaste problem näs via QRC"
        "<li>📱 länk till sida som känner av språkinställning i mobilen och visar info på rätt språk"
        "<li>🍴🏨🛒🚰🚻 länk till sida som visar <b>öppna</b>restauranger, boenden, affärer, dricksvatten, toaletter"
        "</ll>"        ,
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/93",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/97",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/176",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/180",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        ],
        "buzzwords": ["Bilddatabas", "Interoperabilitet", "QR-koder", "Smart tourism", "SSOT","Open data"],
        "stakeholders": ["Tillväxtverket", "Vandrare"]        
    },
    {
        "id": "INAT-1",
        "title": "🐾 SAT – iNaturalist",
        "url": "./179_inat_taxa_layers_colored_5.html",
        "img": "./thumbs/SAT_179_2_iNaturalist_400x225.jpg",
        "desc": "🐾 Datadriven test: på några minuter hämtas communitydriven artdata från iNaturalist och visar vad som faktiskt finns längs leden. Kontrasten mot långsamma processer är tydlig.",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/179",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/146",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148"
        ],
        "buzzwords": ["APIFirst", "Community driven", "Ekosystem","Realtidsdata", "Smart tourism"],
        "stakeholders": ["Community", "Vandrare"]        
    },
    {
        "id": "BBQ-1",
        "title": "🔥🏕️ SAT grillplatser & vindskydd",
        "url": "./map185_vindskydd_grillplatser.html",
        "img": "./thumbs/SAT188_WD_OSM_bbq_400x225.jpg",
        "desc": "🔥🏕️ 8 grillplatser och 11 vindskydd beställdes av Tillväxtverket för 7 miljoner kronor "
        "– men var är de och när levereras de? Kartan visar de som hittats längs leden. 4 har design SAT så minst 7 vindskydd saknas...<br />"
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🧑‍🤝‍🧑🗂️ digital koppling uppdrag, faktura, utbetalt belopp, leverabel. Idag visar Tillväxtverket ingen koppling beställning leverans ..."
        "Ifallet SAT blev det lunch på Grinda och en båttur Utö.. och att Tillväxtverket själva skrev en PDF... SAT själva säger att allt skall vara en överraskning...<br/>"
        "<li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🔥 koppling skall finnas mellan beställning leverans</li>"
        "<li>🔥 projekt som SAT som inte redovisar begär tillbaka pengarna</li>"
        "<li>🛰️ API för felanmälan av alla objekt där objekten skall ha QRC kod, eller objekt 'nära mig'</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder och 'depict'</li></ll>",
        "desc_is_html": True,        
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/49",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/146",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/185"
        ],
        "buzzwords": ["Bilddatabas","Community driven","Interoperabilitet", "Smart tourism", "Open data"],
        "stakeholders": ["Community", "Vandrare"]        
    },
    {
        "id": "FERRY-1",
        "title": "🧭 SAT-leden och Nordsydlinjen",
        "url": "https://umap.openstreetmap.fr/en/map/boende-nordsydlinjen-och-stockholm-archipelago-tra_1257362?scaleControl=true&miniMap=true&scrollWheelZoom=true&zoomControl=true&editMode=disabled&moreControl=true&searchControl=true&tilelayersControl=true&embedControl=null&datalayersControl=true&onLoadPanel=caption&captionBar=true&captionMenus=true&datalayers=35da97a6-893b-46de-b7b3-4ed341b042c3%2Cb3a4a332-36f7-4949-80dd-2468d4a712ea%2C752d2480-e5a5-4fa2-9b78-15c75e545f64%2C7b203db0-53d4-4a1a-9f7d-a298344c6da5&locateControl=true&measureControl=true#8/59.305/18.515",
        "img": "./thumbs/SAT_NordSyd_400x225.jpg",
        "desc": "🧭 Flum som <i>Magic season</i> möter verklighet<br/> SAT-projektet hävdar att NordSyd-linjen ska göra det enkelt att vandra Stockholm Archipelago Trail. <strong>Sanningen</strong>? Nja. Få leder är så splittrade och otydliga som SAT. På kartan ser du hur NordSydlinjen faktiskt går – och var SAT-leden ligger. Och för att krångla till ekvationen ännu mer: SAT vill att vi ska vandra off-season, när NordSydlinjen inte ens går… <br/><b>Resultat:</b> dyra investeringar + flummiga löften = en led som aldrig riktigt hänger ihop. <strong>Tillväxtverket – vakna!</strong> Det handlar om våra skattepengar, som inte skapar verklig nytta utan göder ett osunt bidragstiggeri. Man pratar om “Magic season” och att vi ska “överraskas” – men det vi i praktiken överraskas av är hur oproffsigt skattepengar delas ut till lycksökare som levererar flum istället för resultat.",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/164",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/151",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149"
        ],
        "buzzwords": ["APIFirst", "Ekosystem", "Open data", "Flerspråkighet", "Smart tourism", "SSOT"],
        "stakeholders": [ "Kommuner","Vandrare","Visit Sweden"],     
        "related_videos": [
          {
            "title": "Design för alla Smart turism",
            "url": "https://youtu.be/j-yRPRaTZHU",
            "lang": "sv",
            "comment": "Video som illustrerar konceptet 'Design för alla' och hur smart turism kan bli inkluderande och datadriven. Koppling till tillgänglighet, digital mognad och delad infrastruktur.",        
          }]

    },    
    {
        "id": "NATURE-1",
        "title": "🏕️🌿 SAT – Campsites & Nature Reserves",
        "url": "https://umap.openstreetmap.fr/en/map/stockholm-archipelago-trail-naturreservat-talta_1280785?scaleControl=false&miniMap=false&scrollWheelZoom=false&zoomControl=true&editMode=disabled&moreControl=true&searchControl=null&tilelayersControl=null&embedControl=null&datalayersControl=true&onLoadPanel=none&captionBar=false&captionMenus=true#10/59.0261/18.7015",
        "img": "./thumbs/SAT_Naturreserve_400x225.jpg",
        "desc": "🏕️🌿 Skall man tälta på öarna gäller allemansrätten – "
        "om man inte är i ett naturreservat. 📜 Denna karta visar "
        "Stockholm Archipelago Trail, Naturreservat och länkar till Naturvårdsverkets Skyddad natur och Länsstyrelsens föreskrifter"
        ". Allt detta är på svenska – skall vi <strong>tro att SAT-projektet och Visit Sweden kommer</strong> att skapa en "
        "flerspråkig version av dessa sidor? 😉 se test "
        "<a target=_blank "
        "href='./SAT_195_MachineTranslate_latest.html'>#195 maskin översättning</a>"
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 flerspråkigt data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li></ll>"
        "<br /><br /><b>Interoperabiltet</b> OSM <-> Wikdata <-> Wikicommons",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/164",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/151",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/149"
        ],
        "buzzwords": ["Bilddatabas", "Community driven" ,"Smart tourism", "Open data"],
        "stakeholders": ["Vandrare","Visit Sweden"]        
    },
    {
        "id": "BBQ-2",
        "title": "🏕️🌿 SAT – Grillplatser.nu ",
        "url": "https://grillplatser.nu/Karta/Lan/Stockholms-lan",
        "img": "./thumbs/SAT_grillplatsernu_400x225.jpg",
        "desc": "🏕️🌿 Öppna data har efterfrågats från <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11' target=_blank>SAT 2025-03-19</a> utan svar se hur > 7000 grillplatser "
        "finns samlade av en community med fria bilder. <br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🌍 flerspråkig data</li>"
        "<li>♿ tydligare tillgänglighet</li>"
        "<li>🛰️ API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder</li>"
        "<li>🌍 samma som grillplatser.nu OSM Wikidata</li>"
        "<li>🖼️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "</ll>"
        "<br /><br /><b>Interoperabiltet</b><br/> OSM <-> Wikdata <-> Grillplatser.nu <-> Wikicommons"
        "<ll><li>OSM <a href='https://wiki.openstreetmap.org/wiki/Sv:Key:ref:grillplatser.nu' "
        "target='_blanket'>Key:ref:grillplatser.nu</a></li>"
        "<li>OSM overpass <a href='https://overpass-turbo.eu/s/2cmU' "
        "target='_blanket'>Key:ref:grillplatser.nu</a> </li>"
        "</ll><br />",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/185",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/188"

        ],
        "buzzwords": ["Bilddatabas", "Community driven", "Interoperabilitet",  "Smart tourism", "Open data"],
        "stakeholders": ["Vandrare"]        
    },
    {
        "id": "ECHOSYSTEM-6",
        "title": "🔥 SAT – Wikidata - Wikipedia hur man jobbar datadrivet",
        "url": "https://wikishootme.toolforge.org/#lat=59.0617461708114&lng=18.431606590747837&zoom=12",
        "img": "./thumbs/SAT_Wikishootme_400x225.jpg",
        "desc": "🔥 Öppna data från SAT lovas till <a target_blank href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/49'>Nacka kommun</a> "
        "trovärdigheten är låg när allt är hemligt... ingen hittar vindskydd som utlovats, trappor som byggts..."
        "<br><br><a target=_blank href='https://www.youtube.com/watch?v=A3GnO4kAIos&list=PLNWUKRLAYDeSSHsFOAOy8L_cAtbsOQ41R&index=1'>Video</a> "
        "om hur wikidata i kartan jobbar med bilder, bildbibliotek, > 200 språk versioner. "
        "<br /><br>Idag jobbar Google med <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' target=_blank>Persistenta Identifierare</a> och realtidsinformation med färjetider skall man "
        "prata om <b>Smart turism</b> fungerar det inte bara med filmer på Instagram om <i>Magic season</i>. "
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos</li>"
        "<li>🌍 'samma som' dvs. Linked data </li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och  <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>skapar för vandringsleder det vi ser i Norge/Finland</a></li>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/49",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/143",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/145",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/11",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/161",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171"
        
        ],
        "buzzwords": ["APIFirst","Community driven","Interoperabilitet" , "Smart tourism","SSOT"],
        "stakeholders": ["Community"],
        "related_videos": [
          {
            "title": "Stockholms Archipelago Trails digitala spöke",
            "url": "https://youtu.be/5atWdmH1StA",
            "lang": "🇸🇪 Svenska",
            "comment": "Kritisk genomgång av hur SAT-projektet misslyckats med att leverera öppna data och API:er – kontrasten mot hur Wikidata arbetar öppet och spårbart."
          },
          {
            "title": "SAT: Från PDF till öppen karta",
            "url": "https://youtu.be/n-EQQLI-JTs",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar transformationen från statiska PDF:er till interaktiva kartor med öppen och länkad data – precis det arbetssätt som Wikidata representerar."
          },
          {
            "title": "Kraften i öppen data",
            "url": "https://youtu.be/Tv0Xndq47FM",
            "lang": "🇸🇪 Svenska",
            "comment": "Förklarar hur öppna data skapar värde först när de används och länkas – kärnan i Wikidata och OpenStreetMap-modellen."
          },
          {
            "title": "FAIR Governance – att leda med data",
            "url": "https://youtu.be/8OXiiDjiOYo",
            "lang": "🇸🇪 Svenska",
            "comment": "Förklarar varför offentlig sektor måste styra med FAIR-data och transparens – samma logik som Wikidata bygger på."
          },
          {   
            "title": "Design för alla Smart turism",
            "url": "https://youtu.be/j-yRPRaTZHU",
            "lang": "sv",
            "comment": "Video som illustrerar konceptet 'Design för alla' och hur smart turism kan bli inkluderande och datadriven. Koppling till tillgänglighet, digital mognad och delad infrastruktur.",        
          },
          {
            "title": "Agile Linked Data: Sustainable Trails",
            "url": "https://youtu.be/FW6MDhoCEKk",
            "lang": "🇬🇧 English",
            "comment": "Visar hur agila metoder och länkad data möjliggör hållbar utveckling av leder – med Wikidata som nav för interoperabilitet."
          },
          {
            "title": "Datarevolutionen bakom kartor",
            "url": "https://youtu.be/pvGUv8Bf0Go",
            "lang": "🇸🇪 Svenska",
            "comment": "Kopplar kultur, film och natur genom länkad öppen data – visar praktiskt hur Wikidata och OSM kan integreras i kartor för turism."
          },
          {
            "title": "Make Hiking Data Shine",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇬🇧 English",
            "comment": "Visar hur man gör vandringsdata FAIR och tillgänglig – i kontrast till SAT:s slutna datahantering."
          },
          {
            "title": "The FAIR Trail Network",
            "url": "https://youtu.be/35zAwMobm3o",
            "lang": "🇬🇧 English",
            "comment": "Sätter Wikidata och OpenStreetMap i centrum för en europeisk digital infrastruktur för vandringsleder."
          }
        ]

    },
        {
        "id": "FERRY-2",
        "title": "🔥 SAT – Ferrystops - datadrivet",
        "url": "./187_WD_OSM_ferry_stops_latest.html",
        "img": "./thumbs/SAT_ferry_400x225.jpg",
        "desc": "🔥 Öppna data från SAT dyker inte upp eftersom projektet inte arbetar datadrivet dom tror <i>det är ett vinterjobb</i>. " 
        "Var SAT leden startar och slutar är otydligt utan den knyter ihop bryggor och stigar och är inte en rundslinga. "
        "Den borde vara ett <a target_blank href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/161'>Nodnätverk</a>. "
        "<br />Vi ser hela kedjan är pdf och textdokument från Tillväxtverket till utbetalning av skattepengar... och "
        "slutar med filmer på Instagram med <i>Magic season</i> för att sedan sökas nya bidrag..."
        "<br><b>Google</b> förstår detta med datadrivet och <b>Smart turism</b>. Dom jobbar <b>datadrivet</b> med Google Map och har realtidsinformation om alla "
        " Waxholmsbolagets bryggor och har även alla restauranger med recensioner... "
        "<br />Kartan ovan visar färjestopp som är startpunkter för SAT leden hämtat från <a target_blank href='https://www.youtube.com/watch?v=m_9_23jXPoE'>Wikidata</a>."
        "<br/><br /><b>Önskvärt vore stöd för:</b> "
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ger Tillväxtsverket pengar till datasilos <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder - länkade data</li>"
        "<li>🌍 'samma som' dvs. Linked data </li>"
        "<li>🛰️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och  <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>skapar för vandringsleder det vi ser i Norge/Finland</a></li>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/201",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/39",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/81",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/158",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/186",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/187"
        ],
        "buzzwords": ["APIFirst","Interoperabilitet", "Realtidsdata", "Smart tourism"],
        "stakeholders": ["Kommuner","Skärgårdsstiftelsen", "SSOT","Vandrare","Visit Sweden"],
        "related_videos": [
          {
            "title": "Smart Tourism – Beyond the Brochure",
            "url": "https://youtu.be/35zAwMobm3o",
            "lang": "🇬🇧 English",
            "comment": "Visar hur framtidens turism bygger på öppna data, API:er och realtidsinformation – inte PDF-broschyrer. Direkt koppling till färjestoppens roll i Smart Tourism."
          },        
          {   
            "title": "Design för alla Smart turism",
            "url": "https://youtu.be/j-yRPRaTZHU",
            "lang": "sv",
            "comment": "Video som illustrerar konceptet 'Design för alla' och hur smart turism kan bli inkluderande och datadriven. Koppling till tillgänglighet, digital mognad och delad infrastruktur.",        
          },

          {
            "title": "API first Sweden",
            "url": "https://youtu.be/qN0Cbl8Hsv4",
            "lang": "🇸🇪 Svenska",
            "comment": "Förklarar varför API-först-tänkandet inte slagit igenom i offentlig sektor – en central brist som gör att färjestopp och bryggor saknar realtidsintegration."
          },
          {
            "title": "Datarevolutionen bakom kartor",
            "url": "https://youtu.be/pvGUv8Bf0Go",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur datadrivna kartor kopplar ihop film, kulturarv och natur via Wikidata och OSM – samma metod behövs för SAT:s färjestopp och bryggor."
          },
          {
            "title": "Offentliga leder, öppen data",
            "url": "https://youtu.be/F-Te57TCKzU",
            "lang": "🇸🇪 Svenska",
            "comment": "Lyfter behovet av att offentlig information (som färjestopp) görs tillgänglig via öppna API:er – inte som statiska filer."
          },
          {
            "title": "Den smarta leden",
            "url": "https://youtu.be/mKvdCT2wF94",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur 'Smart Data' kan användas för att skapa dynamiska leder med realtidsstatus – direkt relevant för SAT:s färjetrafik och anslutningar."
          },
          {
            "title": "Från data till destination",
            "url": "https://youtu.be/BOUya4P1UGQ",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hela kedjan från rådata till upplevelse – hur dataflöden om transporter, leder och POI:er kan driva framtidens turism."
          },
          {
            "title": "Open Data Act in Practice",
            "url": "https://youtu.be/QzXfOy7-xJM",
            "lang": "🇸🇪 Svenska",
            "comment": "Belyser hur EU:s öppna datadirektiv kräver att data om mobilitet och transporter publiceras som öppna API:er – exakt det SAT saknar."
          },
          {
            "title": "FAIR Governance – att leda med data",
            "url": "https://youtu.be/8OXiiDjiOYo",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur styrning och uppföljning måste baseras på FAIR-data för transparens – en grundbult för framtidens färje- och ledinformation."
          }
        ]
    },    
        {
        "id": "AED-1",
        "title": "🔥 SAT – AED - Hjärtstartare <-> Hjärtstartarregistret",
        "url": "./191_WD_OSM_AED_latest.html",
        "img": "./thumbs/SAT_AED_191_400x225.jpg",
        "desc": "Hjärtstartare är till viss del uppstyrt av <a href='https://www.hjartstartarregistret.se/#/' target=_blank>Hjärtstartarregistret</a>  men lång ifrån alla apparater finns där. "
        "Jag har inventerat en del genom dels kolla i registret men även pratat med månniskor längs leden och i OSM."
        "<br><ll><li>Bilder på <a href='https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail_AED' target=_blank>Hjärtstartare längs leden</a> "
         "/ <a href='https://wikimap.toolforge.org/?cat=Stockholm_Archipelago_Trail_AED&subcats=true&subcatdepth=3&cluster=false' target=_blank> på en karta</a>"
        "<li>Jag har översatt en polsk app till svenska <a href='https://openaedmap.org/sv/#map=8.89/59.2745/18.942' target=_blank>OpenAED</a> som hämtar sitt data från OSM om AED:er "
        "se issue <a target=_blank href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/46#issuecomment-2820021924'>#46</a>" 
        "<li>Haft diskussion 2025-juni-19 13:30 med <a href='https://www.hjartstartarregistret.se/#/' target=_blank>hjärtstartarregistret</a> om bättre integration med OSM <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/79#issuecomment-2955570965'>#79</a>... "
        "status vi väntar på dom - tveksamt om dom har resurser"
        "<li>Skapat poster i OSM och Wikidata för AED hittade se <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/191'>#191</a>"
        "</lL>"
        "<br><br><b>Utmaningar jag ser</b>"
        "<ll><li>Något som borde vara självklart att ha ordning på drivs av en organisation <a href='https://www.hlr.nu/'>HLR rådet</a>"
        "<li>var i <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/46#issuecomment-2833502212' target=_blank>april i Nice</a> där AED:er finns som staten ansvarar för"
        " känns mycket bättre och <a href='https://openaedmap.org/sv/#map=12.71/43.68708/7.2744&node_id=12902177339' target=_blank>AED:er fanns på strandpromenaden</a>"    
        "<li>Hjärtstartarregistret använder idag Open Street Map kartor och skulle vinna på "
        "bättre integration men är nog inte datadrivna och har små resurser"
        "<li><b>SAT projektet har inte ens en projektyta</b> så vad dom vill berättas ostrukturerat på instagram meddelanden... - galet"
        "<li><b>Naturvårdsverket har projektet sedan 2019 om vandringsleder</b> som definierat när man skall mötas "
        "inte hur data om friluftsliv skall beskrivas... - galet se "
        "<a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/144' target=_blank>ChatGPT om nationell samverkansmodell vandringsutveckling</a>"
        "</lL>"
        "<br />Saker som detta måste styras upp men jag ser ingen som kan detta... "
        "ser inte heller att Tillväxtverket bidrar med krav till Naturvårdsverket om bättre info"
        "<br/><br /><b>Önskvärt vore stöd för:</b>"
        "<ll><li>🧑‍🤝‍🧑🗂️ projektytor och öppna backlogs annars blir det ingen interoperabilitet"
        "<li>🌍 data drivna plattformar och inga datasilos - idag ställer Tillväxtsverket inga krav på koppling hjärtstartare <b>varför</b></li>"
        "<li>🛰️ ETT API för felanmälan</li>"
        "<li>🖼️ koppling till bilddatabaser med fria bilder för hjärtstartare - länkade data</li>"
        "<li>🌍 'samma som' dvs. Linked data där vi skall kunna följa från ansökan - bidrag -leverans inte dagens pdf:er</li>"
        "<li>🛰️ landningssidor för alla leverabler och <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/171' "
        "target='_blanket'>persistenta identifierare</a></li>"
        "<li>🔥 att Naturvårdsverket visar på digital mognad och har med <b>hjärtstartare för friluftslivsdata</b>"
        "</ll>",
        "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/191",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/46",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/144"
        ],
        "buzzwords": ["APIFirst","Interoperabilitet", "Ekosystem" , "Smart tourism", "SSOT"],
        "stakeholders": ["Vandrare"],
        "related_videos": [
          {
            "title": "Från vilse till återfunnen",
            "url": "https://youtu.be/FKco6rs8H3s",
            "lang": "🇸🇪 Svenska",
            "comment": "Tar upp vikten av digital data för att rädda liv – hjärtstartare, positionering och tillgänglig information är centrala delar av säker friluftsturism."
          },
          {
            "title": "Offentliga leder, öppen data",
            "url": "https://youtu.be/F-Te57TCKzU",
            "lang": "🇸🇪 Svenska",
            "comment": "Belyser varför offentligt finansierad information som hjärtstartarlägen ska publiceras som öppna data och integreras via API:er."
          },
          {
            "title": "FAIR Governance – att leda med data",
            "url": "https://youtu.be/8OXiiDjiOYo",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur FAIR-principerna (Findable, Accessible, Interoperable, Reusable) kan användas för att säkerställa ansvar och spårbarhet inom offentlig data – t.ex. Hjärtstartarregistret."
          },
          {
            "title": "Datarevolutionen bakom kartor",
            "url": "https://youtu.be/pvGUv8Bf0Go",
            "lang": "🇸🇪 Svenska",
            "comment": "Demonstrerar hur kultur- och naturdata kan kopplas ihop via Wikidata och OSM – samma metod kan användas för AED:er längs lederna."
          },
          {
            "title": "Den smarta leden",
            "url": "https://youtu.be/mKvdCT2wF94",
            "lang": "🇸🇪 Svenska",
            "comment": "Förklarar hur realtidsdata och smarta system kan skapa säkrare, mer informativa leder med t.ex. varningar och hjärtstartarlägen."
          },
          {
            "title": "Data First – Not Documents",
            "url": "https://youtu.be/GSW9q9u1O2w",
            "lang": "🇬🇧 English",
            "comment": "Visar konsekvenserna när viktiga samhällsdata hålls stängda — förlorad säkerhet, ineffektivitet och dubbelarbete. Relevans: Hjärtstartarregistret som datasilos."
          },
          {
            "title": "Ett brutet löfte om öppna data",
            "url": "https://youtu.be/3oTatm6kiw4",
            "lang": "🇸🇪 Svenska",
            "comment": "Kritisk granskning av SAT-projektets bristande öppna data – direkt parallell till hur livsviktig information (som AED) fortfarande inte delas maskinellt."
          }
        ]       
    },
        {
        "id": "WIKIPEDIA-2",
        "title": "🔥 Wikipedia Recent Changes Map - real time",
        "url": "http://rcmap.hatnote.com/#sv,wikidata,en,uk",
        "img": "./thumbs/WikipediaChangeStream_400x225.jpg",
        "desc": """Exempel hur en community som Wikipedia jobbar med data som data<br/><br/>
        🚵 <b>Behov:</b> Att kunna se ändringar för alla vandringsleder i Sverige<br>
        ❌ <b>Idag:</b> Datasilos där vi inte ens vet vilka leder som finns i Sverige. Enormt frustrerande för vandrare...<br>
        ✅ <b>Önskat:</b> Datadrivet.
        """,
        "desc_is_html": True,
        "buzzwords": ["APIFirst","Interoperabilitet", "Ekosystem" , "Smart tourism"],
        "stakeholders": ["Community","Vandrare"],
        "related_videos": [
          {
            "title": "The Power of the Crowd – Stockholm Archipelago Trail",
            "url": "https://youtu.be/4CkxYAfV_wg",
            "lang": "🇬🇧 English",
            "comment": "Visar hur community-drivna initiativ som OpenStreetMap och Wikidata skapar levande, självförbättrande datanätverk – precis som Wikipedia gör i realtid."
          },
          {
            "title": "Den smarta leden",
            "url": "https://youtu.be/mKvdCT2wF94",
            "lang": "🇸🇪 Svenska",
            "comment": "Fokuserar på hur realtidsdata och öppna API:er kan göra friluftsliv och leder lika dynamiska som Wikipedia – där uppdateringar sker kontinuerligt."
          },
          {
            "title": "Agile Linked Data: Sustainable Trails",
            "url": "https://youtu.be/FW6MDhoCEKk",
            "lang": "🇬🇧 English",
            "comment": "Förklarar hur agila processer och länkad data möjliggör samarbete mellan frivilliga och myndigheter – i linje med Wikipedia-modellen."
          },
          {
            "title": "Datarevolutionen bakom kartor",
            "url": "https://youtu.be/pvGUv8Bf0Go",
            "lang": "🇸🇪 Svenska",
            "comment": "Demonstrerar hur öppen data och Wikidata kan kopplas till kartor, precis som Wikipedia’s realtidsflöde kopplas till platser via Wikidata."
          },
          {
            "title": "Ett brutet löfte om öppna data",
            "url": "https://youtu.be/3oTatm6kiw4",
            "lang": "🇸🇪 Svenska",
            "comment": "Kontrasterar öppenhet och spårbarhet hos Wikipedia med stängda, icke-spårbara offentliga projekt som SAT."
          },
          {
            "title": "Data First – Not Documents",
            "url": "https://youtu.be/GSW9q9u1O2w",
            "lang": "🇬🇧 English",
            "comment": "Visar varför offentliga data måste göras tillgängliga via öppna API:er — inte statiska dokument — för att möjliggöra innovation som Wikipedia gör."
          }
        ]
    },
        {
        "id": "OSM-5",
        "title": "🔥 OSM latest Changes ",
        "url": "https://rene78.github.io/latest-changes/#12/58.9388/18.1906",
        "img": "./thumbs/OSM_changeset_400x225.jpg",
        "desc": """Exempel hur OSM ändringar kan hämtas via en karta<br/>
        🚵 <b>Behov:</b> Att kunna se vem som ändrat vad för vandringsleder i Sverige<br>
        ❌ <b>Idag:</b> Datasilos verkar som Lantmäteriverket inte har ett vandringsleds fokus. Naturvårdsverket har inget APIfirst tänk utan mer likes på Linked in<br>
        ✅ <b>Önskat:</b> Datadrivet
        """,
        "desc_is_html": True,
        "buzzwords": ["APIFirst","Interoperabilitet", "Ekosystem" , "Smart tourism"],
        "stakeholders": ["Community","Vandrare"],
        "related_videos": [
          {
            "title": "The Power of the Crowd – Stockholm Archipelago Trail",
            "url": "https://youtu.be/4CkxYAfV_wg",
            "lang": "🇬🇧 English",
            "comment": "Visar hur crowdsourcing via OSM och Wikidata kan användas för att skapa och förbättra vandringsleder – ett exempel på datadrivet samarbete."
          },
          {
            "title": "Från PDF till API: Ledens framtid",
            "url": "https://youtu.be/cGzB2ufkVeY",
            "lang": "🇸🇪 Svenska",
            "comment": "Belyser varför OSM:s öppna dataflöden bör vara standard istället för PDF-leveranser och slutna rapporter."
          },
          {
            "title": "Den smarta leden",
            "url": "https://youtu.be/mKvdCT2wF94",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur realtidsdata och uppdateringar från OSM kan bli en del av en smart digital infrastruktur för leder."
          },
          {
            "title": "Agile Linked Data: Sustainable Trails",
            "url": "https://youtu.be/FW6MDhoCEKk",
            "lang": "🇬🇧 English",
            "comment": "Förklarar hur öppna datakällor som OSM och Wikidata kan länkas och hållas uppdaterade via gemensamma standarder."
          },
          {
            "title": "Stockholms Archipelago Trails digitala spöke",
            "url": "https://youtu.be/5atWdmH1StA",
            "lang": "🇸🇪 Svenska",
            "comment": "Kritiserar projekt som inte använder öppna, spårbara system som OSM – en kontrast till hur OSM gör ändringar transparenta."
          },
          {
            "title": "Make Hiking Data Shine",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇬🇧 English",
            "comment": "Uppmanar till att göra friluftsdata FAIR – Findable, Accessible, Interoperable, Reusable – precis som OSM:s ändringshistorik redan är."
          },          {
            "title": "CC 0  Nyckeln till datadriven turism",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇸🇪 Svenska",
            "comment": "Hur kan data förändra din semester"
          }

        ]
    },
  {
    "id": "ECHOSYSTEM-7",
    "title": "🌿 Stockholm Archipelago Trail — Amenities by Section - Sankey- Flow Diagrams",
    "url": "./SAT_Sankey.html",
    "img": "./thumbs/SAT248_Sankey_400x225.jpg",            
    "desc": """Visualisering av tillgången till service längs Stockholm Archipelago Trail<br/>
    🚶‍♀️ <b>Syfte:</b> Att visa hur olika typer av faciliteter (toaletter, mat, boende, butiker) fördelar sig mellan öarnas etapper.<br/>
    💡 <b>Insikt:</b> Sankey-diagrammet gör det enkelt att se var service koncentreras — t.ex. fler toaletter på Grinda och fler restauranger på Sandhamn.<br/>
    🧭 <b>Nytta:</b> Förvaltare och vandrare får en översikt över var behovet av kompletterande servicepunkter kan vara störst.<br/>
    📊 <b>Datakälla:</b> OSM + Wikidata, aggregerade efter typ av amenity och etappnamn (SAT_Öar).<br/>
    ✅ <b>Effekt:</b> Gör vandringsleden mer planeringsbar, datadriven och möjlig att optimera för hållbar turism.
    """,
    "desc_is_html": True,
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/248",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/200",
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/247"
        ],
    "buzzwords": ["Datadrivet", "Interoperabilitet", "OpenStreetMap", "Smart tourism", "Visual Analytics"],
    "stakeholders": ["Vandrare", "Kommuner", "Naturvårdsverket", "Destinationsbolag"],
    "related_videos": [
          {
            "title": "Datarevolutionen bakom kartor",
            "url": "https://youtu.be/pvGUv8Bf0Go",
            "lang": "🇸🇪 Svenska",
            "comment": "Visar hur öppna data från Wikidata, OSM och Wikimedia Commons kan kombineras för att skapa datadrivna kartor och flöden — samma princip som i Sankey-visualiseringen."
          },
          {
            "title": "Make Hiking Data Shine",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇬🇧 English",
            "comment": "Om hur data kan visualiseras och göras FAIR – Findable, Accessible, Interoperable, Reusable – för att skapa insikt i mönster och samband."
          },
          {
            "title": "Den smarta leden",
            "url": "https://youtu.be/mKvdCT2wF94",
            "lang": "🇸🇪 Svenska",
            "comment": "Förklarar hur datadriven analys, som Sankey-diagrammet, kan användas för att förstå och förbättra upplevelsen längs en led."
          },
          {
            "title": "Från kaos till smart data",
            "url": "https://youtu.be/NKDyRAtjvDI",
            "lang": "🇸🇪 Svenska",
            "comment": "Belyser steget från ostrukturerade data (t.ex. PDF:er) till strukturerad, visualiserbar data – grunden för den typ av insikter som Sankey-diagrammet ger."
          },
          {
            "title": "Agile Linked Data: Sustainable Trails",
            "url": "https://youtu.be/FW6MDhoCEKk",
            "lang": "🇬🇧 English",
            "comment": "Visar hur sammanlänkad data möjliggör analyser och flödesvisualiseringar som speglar verkliga ekosystem av platser och aktörer."
          },
          {
            "title": "Smart Tourism – Beyond the Brochure",
            "url": "https://youtu.be/35zAwMobm3o",
            "lang": "🇬🇧 English",
            "comment": "Demonstrerar hur visualiseringar av besöksflöden och service kan användas strategiskt i smart turism och destinationsutveckling."
          }
        ]

},{
    "id": "WHEELCHAIR-2",
    "title": "♿ Stockholm Archipelago Trail — Wheelchair Accessibility — Amenities by Section (Sankey Flow Diagram)",
    "url": "./SAT_Sankey_Wheel.html",
    "img": "./thumbs/SAT248_Sankey_Wheel_400x225.jpg",            
    "desc": """Visualisering av tillgänglighet för rullstolsburna längs Stockholm Archipelago Trail<br/>
    ♿ <b>Syfte:</b> Att visa hur tillgängliga faciliteter (toaletter, mat, boende, butiker) fördelar sig mellan öarnas etapper för personer med rullstol.<br/>
    💡 <b>Insikt:</b> Sankey-diagrammet tydliggör var tillgängligheten brister — exempelvis få rullstolsanpassade toaletter på vissa öar, medan andra erbjuder bättre infrastruktur.<br/>
    🧭 <b>Nytta:</b> Förvaltare och beslutsfattare får en överblick över var insatser för ökad tillgänglighet bör prioriteras, medan besökare kan planera sina rutter utifrån faktiska förutsättningar.<br/>
    📊 <b>Datakälla:</b> OpenStreetMap + Wikidata, filtrerat efter attribut som indikerar rullstolsanpassning (“wheelchair=yes”) och aggregerat per etapp (SAT_Öar).<br/>
    ✅ <b>Effekt:</b> Bidrar till en mer inkluderande, datadriven och hållbar turism där fler kan uppleva skärgårdens leder på lika villkor.
    """,
    "desc_is_html": True,
    "issues": [
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/73",
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/248"
    ],
    "buzzwords": ["Tillgänglighet", "Datadrivet", "Interoperabilitet", "OpenStreetMap", "Visual Analytics"],
    "stakeholders": ["Kommuner", "SSOT","Tillgänglighetsdatabasen", "Naturvårdsverket", "Destinationsbolag"],
    "related_videos": [
          {
            "title": "Datarevolutionen bakom kartor",
            "url": "https://youtu.be/pvGUv8Bf0Go",
            "lang": "🇸🇪 Svenska",
            "comment": "Filmen visar hur kultur och natur kan förenas genom datadrivna verktyg. Med utgångspunkt i Saltkråkan 2025 visas hur öppna data från Wikidata, OpenStreetMap och Wikimedia Commons kan koppla film- och teaterplatser till vandringsleder som Stockholm Archipelago Trail. Den demonstrerar “datadriven turism” i praktiken – där berättelser, kulturarv och natur möts på kartan genom länkad öppen data."
          },
          {
            "title": "Smart Tourism – Beyond the Brochure",
            "url": "https://youtu.be/35zAwMobm3o",
            "lang": "🇬🇧 English",
            "comment": "Shows how real-time and linked open data improve planning and accessibility for diverse visitors and travel needs."
          },
          {
            "title": "Make Hiking Data Shine",
            "url": "https://youtu.be/VH1reSTAMwY",
            "lang": "🇬🇧 English",
            "comment": "Explores how FAIR principles — Findable, Accessible, Interoperable, Reusable — can include accessibility attributes to support inclusive design."
          },
        {   
            "title": "Design för alla Smart turism",
            "url": "https://youtu.be/j-yRPRaTZHU",
            "lang": "sv",
            "comment": "Video som illustrerar konceptet 'Design för alla' och hur smart turism kan bli inkluderande och datadriven. Koppling till tillgänglighet, digital mognad och delad infrastruktur.",        
        }

        ]

},
    {
    "id": "ECHOSYSTEM-10",
    "title": "🗺️ Stockholm Archipelago Trail — ESRI Flygfoto med Spår — Jämförande Karta",
    "url": "./SAT260_side_by_side.html",
    "img": "./thumbs/SAT260_side_by_side_400x225.jpg",
    "desc": """Interaktiv jämförande karta som visar Stockholm Archipelago Trail ovanpå ESRI:s flygfoton.<br/>
    🗺️ <b>Syfte:</b> Att ge en realistisk och visuellt rik vy över hur vandringslederna löper genom öar, naturreservat och bebyggda områden.<br/>
    🔍 <b>Insikt:</b> Genom att kombinera satellitfoto och spårdata kan man direkt se terrängtyp, stigar, hamnar och anslutningar till färjelinjer — idealiskt för planering och fältverifiering.<br/>
    🌿 <b>Nytta:</b> Underlättar för kartläggare, naturvårdare och vandrare att kontrollera spårets exakta dragning i relation till verklig miljö och landanvändning.<br/>
    🧩 <b>Datakälla:</b> ESRI World Imagery (flygfoto), OpenStreetMap (spårgeometri via relation 19012437) och Wikidata (metadata per etapp).<br/>
    🚀 <b>Effekt:</b> Ökar transparensen och kvaliteten i kartläggningen av Stockholm Archipelago Trail — ett steg mot bättre dokumenterad och visuellt verifierbar friluftsinfrastruktur.
    """,
    "desc_is_html": True,
    "issues": [
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/260"
    ],
    "buzzwords": ["Flygfoto", "OpenStreetMap", "Visualisering"],
    "stakeholders": ["Lantmäteriet", "Region Stockholm", "Naturvårdsverket", "Friluftsorganisationer"],
    "related_videos": [
          {
            "title": "Datarevolutionen bakom kartor",
            "url": "https://youtu.be/pvGUv8Bf0Go",
            "lang": "🇸🇪 Svenska",
            "comment": "Shows how data from OSM, Wikidata and satellite imagery work together to create transparent, trustworthy maps for planning and outdoor use."
          },
          {
            "title": "Smart Tourism – Beyond the Brochure",
            "url": "https://youtu.be/35zAwMobm3o",
            "lang": "🇬🇧 English",
            "comment": "Connects geospatial visualization and open data to sustainable tourism — turning maps into powerful tools for local development."
          },
          {
            "title": "FAIR Governance – att leda med data",
            "url": "https://www.youtube.com/watch?v=8OXiiDjiOYo",
            "lang": "🇸🇪 Svenska",
            "comment": "Förklarar hur FAIR-principerna (Findable, Accessible, Interoperable, Reusable) bör genomsyra ledning och styrning av offentliga organisationer. Betonar vikten av datakvalitet, metadata, och att följa EU:s Open Data Directive (2023/138)"
          }
        ]

},
    {
    "id": "ECHOSYSTEM-14",
    "title": "🔄 Stockholm Archipelago Trail — Dataflöde & Semantisk Koppling (OSM + Wikidata + Folium)",
    "url": "./SAT258-2025-10-28-052517.png",
    "img": "./thumbs/SAT258_Dataflow_400x225.jpg",
    "desc": """Översiktsdiagram som visar hur data om Stockholm Archipelago Trail flödar mellan OpenStreetMap, Wikidata, Commons och interaktiva Folium-kartor.<br/>
    🧩 <b>Syfte:</b> Att tydliggöra hur semantisk koppling och API-drivet datautbyte möjliggör en enhetlig och uppdateringsbar digital karta över hela leden.<br/>
    🌐 <b>Datakällor:</b> OpenStreetMap (relationer <code>P402</code>, ways <code>P10689</code>), Wikidata (600+ objekt, metadata, SPARQL-frågor) och Wikimedia Commons (>6 000 bilder).<br/>
    ⚙️ <b>Process:</b> Python/Folium-notebooken <code>SAT_ALL_IN_ONE.ipynb</code> hämtar geometri via Overpass API, kombinerar med Wikidata-metadata och exporterar GeoJSON till interaktiva HTML-dashboards.<br/>
    🔁 <b>Feedbackloop:</b> Användargenererade insikter och förbättringar förs tillbaka via GitHub Issues och Facebook-gruppen för att höja datakvaliteten.<br/>
    💡 <b>Nivåstruktur:</b> Från prototyp (nivå 3) till semantisk koppling och SSOT-mål (nivå 5).<br/>
    📈 <b>Effekt:</b> En transparent och återanvändbar modell för datadriven naturturism där öppna källor samverkar för bättre planering, analys och tillgänglighet.
     <b>* Jämför</b> <a href=https://www.youtube.com/watch?v=5atWdmH1StA>Dagens digitala spöke</a>
    """,
    "desc_is_html": True,
    "issues": [
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/258"
    ],
    "buzzwords": ["AI",
        "OpenStreetMap",
        "SSOT"
    ],
    "stakeholders": [
        "Vandrare",
        "Kommuner",
        "Lantmäteriet",
        "Naturvårdsverket",
        "Wikimedia Sverige",
        "OSM-communityt"
    ],
    "related_videos": [
      {
        "title": "Make Hiking Data Shine",
        "url": "https://youtu.be/VH1reSTAMwY",
        "lang": "🇬🇧 English",
        "comment": "Denna film förespråkar att vandringsdata görs FAIR (Findable, Accessible, Interoperable, Reusable). Målet är att uppnå smart turism som drivs av smart data, vilket är nära kopplat till Hiking Maturity Model"
      },
      {
        "title": "Datarevolutionen bakom kartor",
        "url": "https://youtu.be/pvGUv8Bf0Go",
        "lang": "🇸🇪 Svenska",
        "comment": "Filmen visar hur kultur och natur kan förenas genom datadrivna verktyg."
      },
      {
        "title": "Agile Linked Data – Sustainable Trails",
        "url": "https://youtu.be/FW6MDhoCEKk",
        "lang": "🇬🇧 English",
        "comment": "Shows how agile workflows and linked data connect trail segments, images and metadata for transparent geospatial ecosystems."
      },          {
        "title": "CC 0  Nyckeln till datadriven turism",
        "url": "https://youtu.be/VH1reSTAMwY",
        "lang": "🇸🇪 Svenska",
        "comment": "Hur kan data förändra din semester"
      }

        ]

}, {
    "id": "ECHOSYSTEM-15",
    "title": "🚶‍♀️ Stockholm Archipelago Trail — Vägkaraktär & Användbarhet",
    "url": "./SAT299_statistik.html",
    "img": "./thumbs/SAT299statsmall.jpg",
    "desc": """📊 Visar vägkaraktären längs hela <b>Stockholm Archipelago Trail</b> — hur mycket av leden som består av stig, grusväg, landsväg eller bebyggda miljöer.<br/><br/>
    🧭 <b>Syfte:</b> Att synliggöra hur olika vägtyper påverkar upplevelsen, tillgängligheten och naturkänslan på varje etapp.<br/>
    🌿 <b>Insikt:</b> Cirka 45 % av leden består av <b>stig (path)</b>, 17 % av <b>skogs- eller traktorväg (track)</b> och 14 % av <b>väg utan klassificering</b>. Resten är mindre andelar av <b>residential</b>, <b>service</b> och <b>footway</b>.<br/>
    🚶‍♀️ <b>Användbarhet:</b> Genom att klicka på varje etapp visas andelen olika vägtyper, vilket ger en tydlig bild av vandringskvalitet och underlag – från naturstig till grusväg.<br/><br/>
    📈 <b>Datakälla:</b> OpenStreetMap via Overpass API + Wikidata (relationer P402 och ways P10689). Längder och andelar beräknade i Python/Folium.<br/><br/>
    ✅ <b>Effekt:</b> Bidrar till att definiera leden som en <b>verklig led</b> med mätbara egenskaper — inte bara ett streck på kartan — och gör det möjligt att analysera och förbättra sträckor ur ett användarperspektiv.
    """,
    "desc_is_html": True,
    "issues": [
        "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/261"
    ],
    "buzzwords": ["Datadrivet", "Interoperabilitet", "OpenStreetMap", "Friluftsliv", "Smart tourism"],
    "stakeholders": ["Vandrare", "Kommuner", "Naturvårdsverket", "Lantmäteriet"],
    "related_videos": [
        {
            "title": "Från PDF till API: Ledens framtid",
            "url": "https://youtu.be/cGzB2ufkVeY",
            "lang": "sv",
            "comment": "Övergången från statiska PDF-dokument till öppna API:er — samma logik som i denna karta, som hämtar data direkt via Overpass."
        },
        {
            "title": "Design för alla Smart turism",
            "url": "https://youtu.be/j-yRPRaTZHU",
            "lang": "sv",
            "comment": "Video som illustrerar konceptet 'Design för alla' och hur smart turism kan bli inkluderande och datadriven. Koppling till tillgänglighet, digital mognad och delad infrastruktur.",        
        },        
        {
            "title": "Den universella leden – Stockholm Archipelago Trail",
            "url": "https://youtu.be/tjfSc8FdKjo",
            "lang": "sv",
            "comment": "Kartan använder Wikidata-länkar i popuparna, vilket gör leden språköverskridande (language-agnostic)."
        },
        {
            "title": "Att avkoda leden – Stockholm Archipelago Trail",
            "url": "https://youtu.be/VbWErh2A8O8",
            "lang": "sv",
            "comment": "Visualiseringen visar hur leden ‘avkodas’ till strukturerad data enligt OSM/Wikidata-standarder."
        },
        {
            "title": "Länkad data: Kartlägger vår värld",
            "url": "https://youtu.be/3JjOkCv5BeM",
            "lang": "sv",
            "comment": "Visar hur OSM och Wikidata kan länkas samman – exakt det kartan gör för vägkaraktärer."
        },
        {
            "title": "Från kaos till smart data",
            "url": "https://youtu.be/NKDyRAtjvDI",
            "lang": "sv",
            "comment": "Kartan är ett exempel på hur data går från ostrukturerad (PDF) till strukturerad (API och Linked Data)."
        },
        {
            "title": "Hiking Maturity Model",
            "url": "https://youtu.be/SzkKbzsfyRU",
            "lang": "sv",
            "comment": "Kartan representerar mognadsnivå 3–4 i modellen — där data är maskinläsbar, interoperabel och återanvändbar."
        },
        {
            "title": "Stockholm Archipelago Trail – Living Digital Twin",
            "url": "https://youtu.be/JaD7mYzlma8",
            "lang": "sv",
            "comment": "Kartan utgör det geografiska grundlagret i SAT:s digitala tvilling."
        },
        {
            "title": "Stockholm Trail’s Digital Twin (English)",
            "url": "https://youtu.be/uGPEOp3B7WQ",
            "lang": "en",
            "comment": "Engelska versionen av den digitala tvillingens introduktion – kopplar direkt till kartans syfte."
        },
        {
            "title": "Offentliga leder, öppen data",
            "url": "https://youtu.be/F-Te57TCKzU",
            "lang": "sv",
            "comment": "Visar varför data som denna karta bygger på bör vara öppna, maskinläsbara och fria för återanvändning."
        },
        {
            "title": "Gör friluftsäventyr FAIR – Stockholm Archipelago Trail",
            "url": "https://youtu.be/wJwttzo9Ao4",
            "lang": "sv",
            "comment": "Kartan följer FAIR-principerna: Findable, Accessible, Interoperable, Reusable."
        },
        {
            "title": "SAT: Från PDF till öppen karta",
            "url": "https://youtu.be/n-EQQLI-JTs",
            "lang": "sv",
            "comment": "Visar den transformation som denna karta representerar – från statisk dokumentation till öppen, interaktiv data."
        },
        {
            "title": "Vilken kartdata ska vi skapa?",
            "url": "https://youtu.be/-C86emwPTjs",
            "lang": "sv",
            "comment": "Diskuterar just de datalager (vägtyper, reservat, servicepunkter) som denna karta visualiserar."
        }
    ]
}

]




In [8]:
for item in gallery:
    if "related_videos" in item:
        print(f"\n🎬 {item['title']}")
        for video in item["related_videos"]:
            print(f"  - {video['title']}: {video['url']}")


🎬 🌍 SAT - Saltkråkan & den datadrivna skärgården
  - Datarevolutionen bakom kartor: https://youtu.be/pvGUv8Bf0Go
  - Stockholm Archipelago Trail – the Hikers Challenge – The Power of the Crowd: https://youtu.be/4CkxYAfV_wg
  - Skapandet av den moderna leden – Stockholm Archipelago Trail: https://youtu.be/eB3MzlTqo9Q
  - Stockholm Archipelago Trail – Living Digital Twin: https://youtu.be/JaD7mYzlma8
  - Stockholm Trail’s Digital Twin: https://youtu.be/uGPEOp3B7WQ
  - Vilken kartdata ska vi skapa?: https://youtu.be/-C86emwPTjs
  - Smart Tourism – Beyond the Brochure: https://youtu.be/35zAwMobm3o
  - Från policy till stig: https://youtu.be/BOUya4P1UGQ
  - Make Hiking Data Shine: https://youtu.be/VH1reSTAMwY
  - CC 0  Nyckeln till datadriven turism: https://youtu.be/VH1reSTAMwY

🎬 🎥 Video – Stockholms Archipelago Trails digitala spöke
  - Stockholms Archipelago Trails digitala spöke: https://youtu.be/5atWdmH1StA
  - CC 0  Nyckeln till datadriven turism: https://youtu.be/VH1reSTAMwY
  - V

In [9]:
import os

broken = []
for g in gallery:
    for k in ("url", "img"):
        path = g[k].replace("./", "")
        # skip external links
        if path.startswith("http"):
            continue
        if not os.path.exists(path):
            broken.append((g["id"], k, path))

if broken:
    print("⚠️ Missing or mis-cased files:")
    for b in broken:
        print(f"{b[0]:<15} {b[1]:<4} → {b[2]}")
else:
    print("✅ All gallery files exist locally.")


⚠️ Missing or mis-cased files:
VISITSWEDEN-2   url  → SAT300_FilmingLocations.html
VISITSWEDEN-2   img  → thumbs/SAT300_FilmingLocations_400x225.jpg
ECHOSYSTEM      img  → thumbs/SAT_led_small.jpg
AI-1            img  → thumbs/SAT_336_AI_400x225.jpg
DIGITALTMUSEUM-1 img  → thumbs/SAT307_Digitaltmuseum_400x225.jpg
ECHOSYSTEM-1    url  → thumbs/SATInfogrfaph1.png
ECHOSYSTEM-1    img  → thumbs/SATInfogrfaph1_thumb2_small.jpg
ECHOSYSTEM-12   url  → thumbs/OppnaDataMotorn.png
ECHOSYSTEM-12   img  → thumbs/OppnaDataMotorn.png
AI-2            url  → thumbs/AI-Powered_Hiking_Smart_Trail_Guide.png
AI-2            img  → thumbs/AI-Powered_Hiking_Smart_Trail_Guide.png
ECHOSYSTEM-13   url  → Europas_Geospatiala_Transformation_1.pdf
ECHOSYSTEM-13   img  → thumbs/Europas_Geospatiala_Transformation.jpg
ECHOSYSTEM-2    img  → thumbs/SAT_2030_400x225.jpg
ECHOSYSTEM-3    url  → SAT_ALL_IN_ONE_142_3_dashboard_latest.html#UC-1
ECHOSYSTEM-3    img  → thumbs/SAT_UserCases_400x225.jpg
ECHOSYSTEM-4    url  → 

In [10]:
user_cases = [
  {
    "id": 1,
    "title": "Local business owner",
    "buzzwords": ["AI","Economy", "Local growth", "Digital visibility"],
    "desc_is_html": True,
    "desc": """
    🏪 <b>Behov:</b> Synlighet på kartor, koppling till närliggande leder, 
    enkel uppdatering av öppettider och erbjudanden.<br>
    ❌ <b>Idag:</b> Många lokala företag finns inte på OSM/Wikidata 
    eller har felaktig info.<br>
    ✅ <b>Önskat:</b> Enkel onboarding till öppna data + integration med SAT Dashboard 
    så att fler hittar till lokala caféer, gårdsbutiker och vandrarhem.
    """
  },
  {
    "id": 2,
    "title": "Daytrip hikers",
    "buzzwords": ["Accessibility", "AI","Digital twin", "Transport"],
    "desc_is_html": True,
    "desc": """
    🚶‍♀️ <b>Behov:</b> Easy access trails for a day, clear maps, toilets and picnic areas, 
    public transport connections.<br>
    ❌ <b>Idag:</b> Hard to combine SL/ferry schedules with trail planning.<br>
    ✅ <b>Önskat:</b> Integrated map with public transport + SAT entry points.
    """
  },
  {
    "id": 3,
    "title": "Weekend tenting hikers",
    "buzzwords": ["AI","Camping", "Transport"],
    "desc_is_html": True,
    "desc": """
    ⛺ <b>Behov:</b> Places for tents, drinking water, firewood/fireplaces, 
    ferry/boat access for a 2–3 day trip.<br>
    ❌ <b>Idag:</b> Wind shelters and water sources partly missing in OSM/Wikidata.<br>
    ✅ <b>Önskat:</b> Up-to-date camping data, clear rules for fire/tenting in nature reserves.
    """
  },
  {
    "id": 4,
    "title": "Weekend beginner tenting hikers",
    "buzzwords": ["Camping", "Safety", "Transport"],
    "desc_is_html": True,
    "desc": """
    🎒 <b>Behov:</b> Lättillgängliga tältplatser nära färjelägen och service, 
    möjlighet till kortare helgvandringar med “plan B” om vädret blir dåligt.<br>
    ❌ <b>Idag:</b> Svårt att hitta reserverade tältplatser eller skyddade alternativ 
    nära transport och boende.<br>
    ✅ <b>Önskat:</b> Karta som visar “easy camp routes” med backup-stugor, 
    närhet till toalett/vatten och enkel återresa med SL eller Waxholmsbolaget.
    """
  },
  {
    "id": 5,
    "title": "Good food & exclusive stay hikers",
    "buzzwords": ["Accommodation","AI", "Smart tourism", "Transport"],
    "desc_is_html": True,
    "desc": """
    🍷 <b>Behov:</b> Hiking combined with gourmet food, hotels, cabins, 
    high-quality experiences.<br>
    ❌ <b>Idag:</b> Few booking links or curated routes available.<br>
    ✅ <b>Önskat:</b> Packages combining SAT stages with local restaurants and stays.
    """
  },
  {
    "id": 6,
    "title": "Group hiking organizers",
    "buzzwords": ["Interoperability", "Planning"],
    "desc_is_html": True,
    "desc": """
    👥 <b>Behov:</b> Planning for larger groups (school classes, companies), 
    info about shelters, toilets, safety, group permissions.<br>
    ❌ <b>Idag:</b> Trail info is not adapted for group logistics.<br>
    ✅ <b>Önskat:</b> Group-friendly booking & facilities overview.
    """
  },
  {
    "id": 7,
    "title": "Hikers with boat access",
    "buzzwords": ["AI","Transport", "Service"],
    "desc_is_html": True,
    "desc": """
    ⛵ <b>Behov:</b> Combine hiking with private boat access, 
    info on harbors, docks, overnight possibilities.<br>
    ❌ <b>Idag:</b> Transport info split across websites and social media.<br>
    ✅ <b>Önskat:</b> Unified harbor & service map linked to trail sections.
    """
  },
  {
    "id": 8,
    "title": "Non-Swedish speaking visitors",
    "buzzwords": ["Multilingualism", "Smart tourism", "Accessibility"],
    "desc_is_html": True,
    "desc": """
    🌍 <b>Behov:</b> Multilingual maps, translated regulations, 
    booking links for ferries & cabins, clear signage in English/German.<br>
    ❌ <b>Idag:</b> Regulations and maps mostly only in Swedish, booking rarely linked.<br>
    ✅ <b>Önskat:</b> Machine-translated regulations, multilingual Wikidata/OSM labels, 
    integrated booking links.
    """
  },
  {
    "id": 9,
    "title": "MTB (mountain bike) riders",
    "buzzwords": ["Sport", "Transport"],
    "desc_is_html": True,
    "desc": """
    🚵 <b>Behov:</b> Clear rules about where MTB is allowed, trail surfaces, 
    alternative routes.<br>
    ❌ <b>Idag:</b> Few MTB routes tagged in OSM, unclear restrictions in nature reserves.<br>
    ✅ <b>Önskat:</b> MTB-friendly SAT map with allowed/forbidden areas.
    """
  },
  {
    "id": 10,
    "title": "Community sites (ideella föreningar, hembygdsföreningar)",
    "buzzwords": ["Culture", "Community", "Heritage"],
    "desc_is_html": True,
    "desc": """
    🏡 <b>Behov:</b> Visa hembygdsplatser, föreningshus, kulturarv, 
    lokala aktiviteter längs leden.<br>
    ❌ <b>Idag:</b> Spritt på olika små webbsidor, sällan integrerat i större kartor.<br>
    ✅ <b>Önskat:</b> Koppling till Wikidata/OSM så att föreningar 
    kan bli synliga direkt i SAT Dashboard och på turistkartor.
    """
  },
    {
      "id": 11,
      "title": "Commercial tourist sites (Airbnb, glamping, aktivitetsbolag)",
      "buzzwords": ["Commercial", "Tourism", "Smart booking"],
      "desc_is_html": True,
      "desc": """
      🏕️ <b>Behov:</b> Länka boenden och aktiviteter direkt till leden, 
      ex. Airbnb-stugor, kajakuthyrning, guidade turer.<br>
      ❌ <b>Idag:</b> Data ligger låst i plattformar utan öppen länkning.<br>
      ✅ <b>Önskat:</b> Smarta länkar från trail-sektioner till boende/aktiviteter, 
      integration med bokningssystem och öppna API:er.<br>
      📷 <b>Bilder:</b> Fria foton från <a href='https://commons.wikimedia.org/wiki/Category:Tourism_in_Stockholm_Archipelago' target=_blank>Wikimedia Commons</a> kan användas för att illustrera glamping, kajak och turism.
      """
    },
  {
    "id": 12,
    "title": "Dog owners & walkers",
    "buzzwords": ["Pets", "Rules", "Accessibility"],
    "desc_is_html": True,
    "desc": """
    🐕 <b>Behov:</b> Hundrastplatser, vattenställen, regler för hundar i naturreservat, 
    information om var koppelkrav gäller.<br>
    ❌ <b>Idag:</b> Regler skiljer sig mellan reservat, ofta bara på svenska skyltar, 
    och saknas digitalt.<br>
    ✅ <b>Önskat:</b> Enhetlig databas över hundregler + kartlager med hundrastplatser 
    och tillgängliga badplatser för hundar.
    """
  },
  {
    "id": 13,
    "title": "Trail runners",
    "buzzwords": ["Sport", "Facilities", "Timing"],
    "desc_is_html": True,
    "desc": """
    🏃‍♂️ <b>Behov:</b> Dricksvatten längs leden, öppettider för butiker, 
    matställen på båtar, platser att ladda batterier eller lämna packning.<br>
    ❌ <b>Idag:</b> Splittrad info om servicepunkter och varierande öppettider per säsong.<br>
    ✅ <b>Önskat:</b> Realtidskarta med servicepunkter och energipåfyllning 
    längs SAT för löpare och multisportare.
    """
  },
  {
    "id": 14,
    "title": "Accessibility-focused hikers",
    "buzzwords": ["Inclusion", "Accessibility", "Mobility"],
    "desc_is_html": True,
    "desc": """
    ♿ <b>Behov:</b> Leder med hårt underlag, ramper, tillgängliga toaletter, 
    parkeringsplatser och bänkar.<br>
    ❌ <b>Idag:</b> Bristfällig märkning av tillgänglighet i OSM och på skyltar.<br>
    ✅ <b>Önskat:</b> Filterbart kartlager över tillgängliga delsträckor och servicepunkter 
    för personer med nedsatt rörlighet.
    """
  },
  {
    "id": 15,
    "title": "Environmental monitors & researchers",
    "buzzwords": ["Environment", "Citizen science", "Data collection"],
    "desc_is_html": True,
    "desc": """
    🌿 <b>Behov:</b> Samla in och visualisera miljödata (skräp, flora/fauna, erosion) 
    längs leden.<br>
    ❌ <b>Idag:</b> Flera initiativ finns men datan är spridd och inte kopplad till geodata.<br>
    ✅ <b>Önskat:</b> Koppla medborgarforskning till trail-segment via Wikidata/OSM, 
    t.ex. via en SAT-dashboard med observationer.
    """
  },
    {
      "id": 16,
      "title": "Photographers & content creators",
      "buzzwords": ["Media", "Attractions", "Visibility"],
      "desc_is_html": True,
      "desc": """
      📸 <b>Behov:</b> Utsiktspunkter, soluppgång/solnedgångsplatser, 
      vackra fotoobjekt längs leden.<br>
      ❌ <b>Idag:</b> Saknas kuraterade platser och metadata i öppna källor.<br>
      ✅ <b>Önskat:</b> Kartlager med fotoplatser, länkade till Wikimedia Commons och Instagram-taggar.<br>
      🖼️ <b>Bilder:</b> Använd fria bilder från <a href='https://commons.wikimedia.org/wiki/Category:Photography_in_Stockholm_Archipelago' target=_blank>Wikimedia Commons</a> för att inspirera och illustrera fotoplatser.
      """
    },
  {
    "id": 17,
    "title": "Bird watchers & nature enthusiasts",
    "buzzwords": ["Wildlife", "Conservation", "Tourism"],
    "desc_is_html": True,
    "desc": """
    🦉 <b>Behov:</b> Information om fågeltorn, säsongsrestriktioner och observationer.<br>
    ❌ <b>Idag:</b> Få OSM-taggar för ornitologiska platser, svårt att hitta aktuella observationer.<br>
    ✅ <b>Önskat:</b> Länkning till Artportalen och naturvårdsdata via Wikidata-ID för varje naturreservat.
    """
  },
  {
    "id": 18,
    "title": "Digital nomads / remote workers",
    "buzzwords": ["Connectivity", "Workation", "Smart tourism"],
    "desc_is_html": True,
    "desc": """
    💻 <b>Behov:</b> Wifi, laddstationer, caféer med eluttag, tysta platser för arbete.<br>
    ❌ <b>Idag:</b> Ingen samlad info om digital infrastruktur i skärgården.<br>
    ✅ <b>Önskat:</b> Kartlager med “digitala oaser” längs SAT, integrerat med kollektivtrafik och logi.
    """
  },
  {
    "id": 19,
    "title": "Active seniors & healthy aging hikers",
    "buzzwords": ["Health", "Community", "Wellbeing"],
    "desc_is_html": True,
    "desc": """
    🧓 <b>Behov:</b> Trygga leder, kortare delsträckor, rastplatser och toaletter, 
    sociala aktiviteter och naturgym.<br>
    ❌ <b>Idag:</b> Svårt att se vilka sträckor som är lättgångna eller har sittplatser.<br>
    ✅ <b>Önskat:</b> Karta med “seniorvänliga” leder, tillgängliga toaletter, och tips på organiserade gruppvandringar.
    """
  },
  {
    "id": 20,
    "title": "Families with children",
    "buzzwords": ["Family", "Education", "Accessibility"],
    "desc_is_html": True,
    "desc": """
    👨‍👩‍👧‍👦 <b>Behov:</b> Kortare etapper, lekvänliga platser, säkra bad, fik och boenden 
    anpassade för barn.<br>
    ❌ <b>Idag:</b> Information om barnvänliga delar saknas på de flesta kartor.<br>
    ✅ <b>Önskat:</b> Filter för barnvänliga sträckor, badplatser och familjeaktiviteter.
    """
  },
  {
    "id": 21,
    "title": "Solo travelers & singles",
    "buzzwords": ["Community", "Freedom", "Wellbeing"],
    "desc_is_html": True,
    "desc": """
    💫 <b>Behov:</b> Trygga leder, sociala mötesplatser, enkel logistik för enpersonsturer.<br>
    ❌ <b>Idag:</b> De flesta guider utgår från par eller grupper.<br>
    ✅ <b>Önskat:</b> SAT Dashboard med tips på “solo friendly” leder, gemensamma vandringar och delade boenden.
    """
  },
  {
    "id": 22,
    "title": "Culture & heritage explorers",
    "buzzwords": ["Culture", "History", "Education"],
    "desc_is_html": True,
    "desc": """
    🏛️ <b>Behov:</b> Kulturmiljöer, fornminnen, museer och berättelser kopplade till skärgårdens historia.<br>
    ❌ <b>Idag:</b> Kulturdata är utspridd på flera plattformar och saknar geografisk koppling.<br>
    ✅ <b>Önskat:</b> Karta med kulturstopp via Wikidata och Riksantikvarieämbetet.
    """
  },
  {
    "id": 23,
    "title": "Eco-conscious travelers",
    "buzzwords": ["Sustainability", "Climate", "Slow travel"],
    "desc_is_html": True,
    "desc": """
    🌱 <b>Behov:</b> Miljövänliga transporter, ekologiska matställen, 
    återvinningsstationer och naturvänliga övernattningar.<br>
    ❌ <b>Idag:</b> Hållbarhetsdata saknas ofta i kartplattformar.<br>
    ✅ <b>Önskat:</b> SAT-karta med klimatvänliga färjealternativ, elbåtsladdning och eko-caféer.
    """
  },
  {
    "id": 24,
    "title": "Wellness & retreat travelers",
    "buzzwords": ["Wellbeing", "Mindfulness", "Nature therapy"],
    "desc_is_html": True,
    "desc": """
    🧘 <b>Behov:</b> Platser för stillhet, yoga i naturen, bad och återhämtning.<br>
    ❌ <b>Idag:</b> Få digitala resurser lyfter fram tysta naturupplevelser.<br>
    ✅ <b>Önskat:</b> Karta med “wellness spots” – tysta vikar, badställen och retreats.
    """
  },
  {
    "id": 25,
    "title": "Educational groups & field trips",
    "buzzwords": ["Learning", "Education", "Youth"],
    "desc_is_html": True,
    "desc": """
    🧭 <b>Behov:</b> Platser för biologi/geografi/exkursioner, material för skolor, säkerhetsinfo och logistik.<br>
    ❌ <b>Idag:</b> Få kopplingar mellan skolprojekt och naturreservatsdata.<br>
    ✅ <b>Önskat:</b> API och kartlager som skolor kan använda för exkursioner med realtidsdata.
    """
  },
    {
      "id": 26,
      "title": "Författare av reseböcker",
      "buzzwords": ["Publishing", "Storytelling", "Curation"],
      "desc_is_html": True,
      "desc": """
      📚 <b>Behov:</b> Tillgång till uppdaterade, pålitliga data om leder, 
      platser, kulturmiljöer och logi för att skapa reseböcker, bloggar och guider.<br>
      ❌ <b>Idag:</b> Saknar central källa för öppna och verifierade fakta.<br>
      ✅ <b>Önskat:</b> Ett “SAT knowledge hub” med Wikidata + OSM som grund för inspiration och research.<br>
      📖 <b>Bilder:</b> Fria kartor och fotografier från 
      <a href='https://commons.wikimedia.org/wiki/Category:Maps_of_Stockholm_Archipelago' target=_blank>Wikimedia Commons</a> 
      kan användas i reseböcker och guider jmf <a href='https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail' target=_blank>bilder från leden</a> 
      """
    },
  {
    "id": 27,
    "title": "Trip sharers & trail community",
    "buzzwords": ["Community", "Open data", "Experience sharing"],
    "desc_is_html": True,
    "desc": """
    🌐 <b>Behov:</b> Kunna dela vandringar, erfarenheter, bilder och tips på ett öppet sätt 
    — utan att allt försvinner i sociala medier-flöden.<br>
    ❌ <b>Idag:</b> De flesta delar via Facebook, svårt att länka eller återanvända.<br>
    ✅ <b>Önskat:</b> "Share Your Hike" <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/151' target=_blank>#151</a> 
    – öppet system där man kan dela turer med spår, bilder, väder och feedback.
    """
  },
  {
    "id": 28,
    "title": "Naturvårdsverket",
    "buzzwords": ["Policy", "Environment", "Data governance"],
    "desc_is_html": True,
    "desc": """
    🏞️ <b>Behov:</b> Nationell överblick över leder, naturreservat, servicepunkter 
    och tillgänglighet kopplad till friluftsmålen.<br>
    ❌ <b>Idag:</b> Uppföljning kräver manuell inrapportering från län och kommuner.<br>
    ✅ <b>Önskat:</b> En gemensam datastandard (OSM/Wikidata) för leder och servicepunkter 
    som kan användas för automatiserad rapportering till miljö- och friluftsmål.
    """
  },
  {
    "id": 29,
    "title": "Myndighetsnätverket för friluftsliv",
    "buzzwords": ["Coordination", "Open data", "Sustainability"],
    "desc_is_html": True,
    "desc": """
    🧭 <b>Behov:</b> Samordnad informationsstruktur mellan myndigheter som arbetar med friluftsliv, 
    naturvård, hälsa och tillgänglighet.<br>
    ❌ <b>Idag:</b> Varje myndighet har egna register och definitioner.<br>
    ✅ <b>Önskat:</b> En nationell modell för "Friluftslivsdata som data" – 
    gemensamt schema för leder, service och naturdata som kan återanvändas.
    """
  },
  {
    "id": 30,
    "title": "Boverket",
    "buzzwords": ["Planning", "Accessibility", "Urban nature"],
    "desc_is_html": True,
    "desc": """
    🏙️ <b>Behov:</b> Data om tätortsnära natur, tillgänglighet och friluftsområden 
    för fysisk planering och rapportering mot friluftsmålen.<br>
    ❌ <b>Idag:</b> Kommuner rapporterar i olika format, svår att jämföra nationellt.<br>
    ✅ <b>Önskat:</b> En gemensam datastandard som binder ihop leder, parker, 
    skyddade områden och tillgänglighetsdata via Wikidata/OSM.
    """
  },
  {
    "id": 31,
    "title": "Folkhälsomyndigheten",
    "buzzwords": ["Health", "Equity", "Urban design"],
    "desc_is_html": True,
    "desc": """
    ❤️ <b>Behov:</b> Underlag för fysisk aktivitet och naturens roll för folkhälsa, 
    särskilt för utsatta grupper.<br>
    ❌ <b>Idag:</b> Data om tillgång till friluftsliv saknas i många hälsoundersökningar.<br>
    ✅ <b>Önskat:</b> Länkade dataset mellan naturtillgång, leder, tillgänglighet 
    och socioekonomiska data för jämlikhetsanalys.
    """
  },
  {
    "id": 32,
    "title": "Havs- och vattenmyndigheten",
    "buzzwords": ["Marine", "Conservation", "Outdoor recreation"],
    "desc_is_html": True,
    "desc": """
    🌊 <b>Behov:</b> Identifiera och följa upp kustnära områden av riksintresse för friluftsliv.<br>
    ❌ <b>Idag:</b> Data om kustleder och badplatser saknar standardiserad koppling till skyddade områden.<br>
    ✅ <b>Önskat:</b> Länkning mellan marina friluftsområden, Wikidata och OSM 
    för bättre analys av tillgång till havsnära rekreation.
    """
  },
  {
    "id": 33,
    "title": "Jordbruksverket",
    "buzzwords": ["Landscape", "Biodiversity", "Rural tourism"],
    "desc_is_html": True,
    "desc": """
    🌾 <b>Behov:</b> Data om leder, ängs- och hagmarker och kulturmiljöer 
    för att följa upp miljömålet “Ett rikt odlingslandskap”.<br>
    ❌ <b>Idag:</b> Svårt att kombinera friluftsliv och jordbrukslandskap i GIS och analyser.<br>
    ✅ <b>Önskat:</b> Länkad data mellan naturvärden, kulturmiljöer, leder och besöksnäring.
    """
  },
  {
    "id": 34,
    "title": "Konsumentverket",
    "buzzwords": ["Safety", "Tourism", "Oversight"],
    "desc_is_html": True,
    "desc": """
    🧾 <b>Behov:</b> Säkerhet och kvalitet i naturturism och friluftstjänster 
    (guider, boenden, aktiviteter).<br>
    ❌ <b>Idag:</b> Ingen enhetlig struktur för att koppla säkerhetsinformation 
    till friluftsdestinationer.<br>
    ✅ <b>Önskat:</b> Standardiserad metadata för säkerhet och certifiering 
    kopplad till Wikidata/OSM-objekt.
    """
  },
  {
    "id": 35,
    "title": "Myndigheten för delaktighet (MFD)",
    "buzzwords": ["Accessibility", "Inclusion", "Digitalization"],
    "desc_is_html": True,
    "desc": """
    ♿ <b>Behov:</b> Att identifiera och synliggöra tillgängliga leder och platser 
    för personer med funktionsnedsättning.<br>
    ❌ <b>Idag:</b> Olikartad och manuell rapportering, saknas samlad digital karta.<br>
    ✅ <b>Önskat:</b> Nationell datamodell för tillgänglighet i natur- och friluftsliv 
    med öppna API:er och gemensamma attribut.
    """
  },
  {
    "id": 36,
    "title": "Länsstyrelsen Stockholm",
    "buzzwords": ["Regional coordination", "Nature protection", "Outdoor recreation"],
    "desc_is_html": True,
    "desc": """
    🗺️ <b>Behov:</b> Samla regional data om leder, naturreservat, badplatser, toaletter och skyddade områden.<br>
    ❌ <b>Idag:</b> Informationen finns spridd mellan GIS-system, broschyrer och webbplatser.<br>
    ✅ <b>Önskat:</b> Gemensam datamodell för friluftsliv som kan exporteras till Wikidata, OSM och nationella karttjänster.
    """
  },
  {
    "id": 37,
    "title": "Tillväxtverket",
    "buzzwords": ["Economy", "Tourism", "Sustainability"],
    "desc_is_html": True,
    "desc": """
    📈 <b>Behov:</b> Stödja hållbar turism och lokal tillväxt genom tillgängliga data 
    om natur- och kulturturism.<br>
    ❌ <b>Idag:</b> Svårt att mäta och analysera naturturismens infrastruktur nationellt.<br>
    ✅ <b>Önskat:</b> Enhetlig dataplattform för turismens infrastruktur (leder, boende, aktiviteter) 
    med öppna standarder.
    """
  },
  {
    "id": 38,
    "title": "Kommuner längs leden",
    "buzzwords": ["Local planning", "Tourism", "Sustainability"],
    "desc_is_html": True,
    "desc": """
    🏘️ <b>Behov:</b> Underlag för lokal planering av leder, servicepunkter och besöksmål.<br>
    ❌ <b>Idag:</b> Olika format och rutiner mellan kommuner, svårt att samordna med länsstyrelsen.<br>
    ✅ <b>Önskat:</b> Lokal tillgång till samma öppna friluftslivsdata – 
    så att uppdateringar i kommunen direkt syns i nationella kartor och Wikidata.
    """
  },
  {
    "id": 39,
    "title": "Reseinfluencers & media",
    "buzzwords": ["Media", "Outreach", "Tourism"],
    "desc_is_html": True,
    "desc": """
    📱 <b>Behov:</b> Tillgång till fria bilder, fakta och kartlänkar om leder och platser 
    för att skapa trovärdigt innehåll.<br>
    ❌ <b>Idag:</b> Spritt material, svårt att hitta fria bilder och källor.<br>
    ✅ <b>Önskat:</b> Enkel åtkomst till fria bilder på Wikimedia Commons 
    och länkbar data för artiklar och sociala medier.
    """
  },
  {
    "id": 40,
    "title": "Vintercampare",
    "buzzwords": ["Winter", "Camping", "Survival"],
    "desc_is_html": True,
    "desc": """
    ❄️ <b>Behov:</b> Tillgång till vatten, ved och skyddade tältplatser även under vintersäsongen.<br>
    ❌ <b>Idag:</b> Vattenposter och vindskydd saknar säsongsdata och metadata i OSM/Wikidata.<br>
    ✅ <b>Önskat:</b> Datamodell med säsongsöppettider och tillgångsstatus för vinterfriluftsliv.
    """
  },
    {
  "id": 41,
  "title": "Trail planners & maintainers",
  "buzzwords": ["Infrastructure", "Planning", "Data quality"],
  "desc_is_html": True,
  "desc": """
  🧭 <b>Behov:</b> Helhetsbild över befintliga och planerade leder, 
  tillgång till data om dricksvatten, toaletter, tillgänglighet, badplatser, hundrastgårdar, 
  hotell och tältplatser med faciliteter.<br>
  ❌ <b>Idag:</b> Information spridd i olika källor (kommuner, OSM, broschyrer), 
  saknar gemensam struktur och uppdateringsflöde.<br>
  ✅ <b>Önskat:</b> En “Trail Infrastructure Dashboard” som samlar 
  OSM/Wikidata-data om leder, servicepunkter och tillgänglighet. 
  Möjlighet att planera, jämföra och kvalitetssäkra ledsträckningar digitalt 
  samt exportera till GIS och kommunala planeringssystem.<br>
  🛠️ <b>Exempel på användning:</b>  
  • Identifiera var nya leder kan dras i relation till befintlig infrastruktur.<br>
  • Analysera servicegap (t.ex. var toaletter eller vattenposter saknas).<br>
  • Dokumentera tillgänglighetsanpassning och planera förbättringar.<br>
  • Samarbeta med kommuner, föreningar och OSM-communityn för uppdateringar.
  """
},{
  "id": 42,
  "title": "Friluftskartskapare & GIS-utvecklare",
  "buzzwords": ["Mapping", "Data quality", "Open data", "Collaboration", "Infrastructure"],
  "desc_is_html": True,
  "desc": """
  🗺️ <b>Behov:</b> Tillgång till stabila, uppdaterade datakällor för leder, servicepunkter, naturreservat och regler, 
  samt tydliga identifierare (Q-id, OSM-id) för synkronisering mellan plattformar.<br>
  ❌ <b>Idag:</b> Kartskapare måste kombinera data från flera källor (OSM, Wikidata, kommunala GIS) 
  med varierande kvalitet och uppdateringsfrekvens. Det är svårt att veta vad som är den senaste versionen 
  av en led eller ett naturreservat, och dialogen med ledägare sker ofta manuellt.<br>
  ✅ <b>Önskat:</b>  
  • Stabil och versionssäkrad databas för friluftsliv som kan användas i appar, tryckta kartor, uMap och dashboards.<br>
  • Enkelt att <b>kommunicera med ledägare</b> (t.ex. via Wikidata-länkar eller SAT Dashboard feedback).<br>
  • Smidig <b>felrapportering</b> kopplad till OSM-objekt eller Wikidata-ID, 
    med notifiering till ansvariga organisationer.<br>
  • Koppling till öppna API:er och standarder (OSM/Wikidata) för enkel uppdatering och återanvändning.<br>
  🧩 <b>Exempel på användning:</b>  
  • Bygga uMap-, Folium- eller Leaflet-kartor med levande friluftsdata.<br>
  • Exportera data till GIS-format (GeoJSON, shapefile) för analys och tryck.<br>
  • Samköra OSM-geometrier (relationer/vägar) med Wikidata-metadata.<br>
  • Identifiera saknade objekt eller felaktiga taggar i OSM/Wikidata via dashboardar.<br>
  💬 <b>Kommunikation:</b>  
  • Möjlighet att lämna feedback direkt på kartobjekt (t.ex. via GitHub Issues eller integrerat rapporteringsverktyg).<br>
  • Länk mellan varje ledsektion och dess förvaltare för dialog och statusuppdateringar.<br>
  💡 <b>Relevanta initiativ:</b>  
  <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail' target=_blank>Stockholm Archipelago Trail (SAT)</a>, 
  <a href='https://github.com/osmse/open-data' target=_blank>OSM Sverige Open Data</a>, 
  samt <a href='https://wikidata.org/wiki/Wikidata:WikiProject_Outdoor_recreation' target=_blank>Wikidata:Outdoor recreation project</a>.
  """
}, {
  "id": 43,
  "title": "Ledägare & förvaltare",
  "buzzwords": ["Maintenance", "Collaboration", "Data governance", "Reporting"],
  "desc_is_html": True,
  "desc": """
  🥾 <b>Behov:</b> Enkel hantering av felrapporter, uppdateringar och dialog med kartskapare, kommuner och användare. 
  Möjlighet att se status, statistik och förändringar över tid för egna leder och servicepunkter.<br>
  ❌ <b>Idag:</b> Uppdateringar sker manuellt via e-post eller PDF-kartor. 
  Det saknas gemensam digital kanal för rapportering och uppföljning. 
  Många fel (trasiga skyltar, stängda toaletter, saknad markering) når aldrig rätt mottagare.<br>
  ✅ <b>Önskat:</b>  
  • Ett digitalt gränssnitt (t.ex. “SAT Dashboard for Maintainers”) kopplat till OSM/Wikidata där felrapportering, status och åtgärder loggas.<br>
  • Möjlighet att verifiera eller avvisa rapporter direkt i systemet.<br>
  • Automatisk notifiering när nya rapporter kommer in på egna leder eller områden.<br>
  • Enkel export till kommunala GIS-system och uppföljningsrapporter.<br>
  🧭 <b>Exempel på användning:</b>  
  • Se vilka sektioner som har aktuella problem (vindfällen, trasiga spänger, saknade skyltar).<br>
  • Godkänna eller uppdatera information som skickats in av kartskapare eller vandrare.<br>
  • Dela uppdaterad status till Wikidata och OSM för omedelbar spridning.<br>
  💬 <b>Kommunikation:</b>  
  • Direktkanal till kartskapare, kommuner och myndigheter via öppna verktyg (GitHub Issues, Wikidata kommentarer, API-feedback).<br>
  • Möjlighet att lägga in kontaktuppgifter till förvaltare per ledsektion i Wikidata (t.ex. via egenskap <code>P127</code> eller <code>P137</code>).<br>
  🌍 <b>Relevanta initiativ:</b>  
  <a href='https://github.com/salgo60/Stockholm_Archipelago_Trail' target=_blank>Stockholm Archipelago Trail (SAT)</a>, 
  <a href='https://wiki.openstreetmap.org/wiki/Sweden' target=_blank>OpenStreetMap Sverige</a>, 
  samt <a href='https://www.wikidata.org/wiki/Wikidata:WikiProject_Protected_areas' target=_blank>Wikidata:Protected areas</a>.
  """
}



]



In [11]:
intro_html = _render_intro_html("""
<!-- Hero-banner klickbar -->
<section id="hero"
  onclick="expandImage()"
  title="Klicka för att förstora"
  style="
    position:relative;
    background: url('OpenDataMotorniSverigesFriluftsliv.png');
    background-size: cover;
    background-position: center;
    height: 260px;
    border-radius:10px;
    margin-bottom:16px;
    cursor: zoom-in;
    box-shadow:0 2px 8px rgba(0,0,0,0.25);">

  <!-- Hint -->
  <div style="
    position:absolute;
    bottom:8px;
    right:12px;
    background:rgba(0,0,0,0.55);
    color:white;
    padding:4px 8px;
    border-radius:6px;
    font-size:0.8rem;">
    🔍 Klicka för förstoring
  </div>
</section>

<!-- Fullscreen overlay -->
<div id="overlay"
  onclick="collapseImage()"
  style="
    display:none;
    position:fixed;
    inset:0;
    background:rgba(0,0,0,0.85);
    z-index:9999;
    align-items:center;
    justify-content:center;
  ">

  <img src="OpenDataMotorniSverigesFriluftsliv.png"
       style="
         max-width:95%;
         max-height:95%;
         border-radius:8px;
         box-shadow:0 10px 40px rgba(0,0,0,0.6);
         cursor: zoom-out;
       ">

  <div onclick="collapseImage()"
    style="
      position:absolute;
      top:16px;
      right:20px;
      font-size:32px;
      color:white;
      cursor:pointer;">
    ✕
  </div>
</div>

<!-- Textsektion under bilden -->
<section style="
  font-family:system-ui,Segoe UI,Roboto,sans-serif;
  padding: 8px 4px 0 4px;">
  <h2 style="margin:0 0 8px 0; font-size:1.5rem; font-weight:700;">
    🧭 Proof by Doing – SAT Open Data – Stockholm Archipelago Trail
  </h2>
  <p style="margin:4px 0; font-size:1rem; font-weight:500;">
    Den här dashboarden visar vad som blir möjligt när fysiska upplevelser möter öppen data.<br>
    Kommuner, myndigheter och föreningar är välkomna att återanvända datan via 
    <a href="https://www.wikidata.org/wiki/Wikidata:WikiProject_Stockholm_Archipelago_Trail" 
       target="_blank" style="color:#2563eb; text-decoration:none; font-weight:600;">
       Wikidata</a> – jmf video om 
    <a href="https://www.youtube.com/watch?v=5atWdmH1StA">dagens digitala spöke</a> – 
    <a href="https://www.youtube.com/playlist?list=PLNWUKRLAYDeTeJjC4RklcS0M_Kp_1htMd">fler filmer</a><br><br>

    💡 Genom att skapa en 
    <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/243">
      digital tvilling</a> 
    av vandringsleden kan öppna data, kartor, 
    <a href="https://en.wikipedia.org/wiki/Single_source_of_truth">SSOT</a> 
    och användarfeedback vävas samman till ett levande beslutsstöd för både vandrare och förvaltare – 
    <a href="https://www.youtube.com/watch?v=NKDyRAtjvDI">
      video ”Från kaos till smart data”
    </a>.
  </p>
</section>

<!-- JS för expand/collapse -->
<script>
function expandImage() {
  document.getElementById("overlay").style.display = "flex";
}
function collapseImage() {
  document.getElementById("overlay").style.display = "none";
}
// ESC för stäng
document.addEventListener("keydown", function(e) {
  if (e.key === "Escape") collapseImage();
});
</script>
""")


In [12]:
import pandas as pd
from datetime import datetime
import time,os,sys,re
from pathlib import Path 
import shutil

OUTPUT_DIR   = "./output"  # ändra vid behov
PROJECT_NAME = "SAT_ALL_IN_ONE_142_3"
stamp        = pd.Timestamp.now().strftime("%Y%m%d_%H%M")

print(stamp)


dash_html = make_sat_dashboard(
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        stamp=stamp,
        intro_html=intro_html,
        map_gallery=gallery,
        user_cases=user_cases
    )
print("✅ Dashboard with gallery =>", dash_html)

20260104_1442
Saved (timestamped): output/SAT_ALL_IN_ONE_142_3_dashboard_20260104_144257.html
Updated (latest): output/dashboard.html
Current path: /Users/salgo/Documents/GitHub/Stockholm_Archipelago_Trail/Notebook
✅ Dashboard with gallery => output/SAT_ALL_IN_ONE_142_3_dashboard_20260104_144257.html


In [13]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2026-01-04 14:42:57
Total time elapsed: 00 minutes 00.71 seconds
